In [1]:
!pip install modal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.6/552.6 kB 14.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for grpclib: filename=grpclib-0.4.7-py3-none-any.whl size=76219 sha256=7d24c14a4f903cd9f6a9317621c62ea7799b2eb1708c54d7f19d28696cf9bff1
  Stored in directory: /root/.cache/pip/wheels/05/c0/1c/3d807409d0c67efeab2949832ba409205b1b6fe03f739ae4c1
Successfully built grpclib


In [6]:
!rm -rf /kaggle/working/*

In [ ]:
!modal token set --token-id ak-SeA6mwfp0axNqaR2IMEQH3 --token-secret as-2cut1gZPDlf5MWMnSV0F6S 

In [3]:
!modal token set --token-id ak-SeA6mwfp0axNqaR2IMEQH3 --token-secret as-2cut1gZPDlf5MWMnSV0F6S 

Verifying token against https://api.modal.com
Token verified successfully!
⠋ Storing token
Token written to /root/.modal.toml in profile upproj.


In [7]:
!modal volume get my-volume checkpoint_epoch_0_batch_301799/

⠋ Downloading file(s) to local...
⠸ Downloading file(s) to local...0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ .
⠦ Downloading file(s) to local...0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ .
⠏ Downloading file(s) to local...0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ .
checkpoint_epoch_0_batch_301799/training_state.pt ━━━━━ 0.0% • 0.0… • ? • -:--:…
                                                               MB               
     checkpoint_epoch_0_batch_301799/scheduler.pt ━━━━━ 0.0% • 0.0… • ? • -:--:…
                                                               kB               
     checkpoint_epoch_0_batch_301799/optimizer.pt ━━━━━ 0.0% • 0.0… • ? • -:--:…
                                                               GB               
checkpoint_epoch_0_batch_301799/model.safetensors ━━━━━ 0.0% • 0.0… • ? • -:--:…
⠹ Downloading file(s) to local...
checkpoint_epoch_0_batch_301799/training_state.pt ━━━━━ 0.0% • 0.0… • ? • -:--:…
                                                        

In [3]:

# Define the content to be written
content = r'''
import modal
import os
import torch
import torch.optim as optim
from torch.utils.cpp_extension import load
from tqdm import tqdm
import math
from transformers import GPT2Tokenizer
from datasets import load_dataset

cuda_version = "12.4.0"  # should be no greater than host CUDA version
flavor = "devel"  # includes full CUDA toolkit
os_version = "ubuntu22.04"
tag = f"{cuda_version}-{flavor}-{os_version}"
volume = modal.Volume.from_name("my-volume", create_if_missing=True)

# Define the Modal stub and image
app = modal.App("complex_model_stub")
image = modal.Image.debian_slim().from_registry(f"nvidia/cuda:{tag}", add_python="3.11").pip_install(
    "torch", "transformers", "datasets==2.20.0", "einops", "tqdm", "ninja", "packaging", "wheel", "zstandard", "numba", "textstat"
)

# Specify GPU, custom image, and volume mount requirements
@app.function(image=image, gpu="t4", timeout=86400, volumes={"/checkpoints": volume})
async def run_training():
    import torch
    from numba import cuda, float64
    import math
    import numpy as np
    import textstat
    import torch
    import torch.nn.functional as F
    from torch.distributions import Categorical

    # Ensure that Numba uses the same device as PyTorch
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # -------------------------------
    # CUDA Kernels
    # -------------------------------

    @cuda.jit
    def fused_scan_fwd_kernel(u, delta, A, B, C, D, output, b, l, d_in, n):
        """
        Forward pass kernel implementing Blelloch's inclusive scan.
        """
        shared_mem = cuda.shared.array(shape=512, dtype=float64)
        input_x = shared_mem[:256]
        scan_x = shared_mem[256:]

        idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        total = b * l * d_in

        if idx >= total:
            return

        dim = idx % d_in
        seq = (idx // d_in) % l
        batch = idx // (l * d_in)

        u_val = u[batch, seq, dim]
        delta_val = delta[batch, seq, dim]

        # Initialize scan_x to zero
        for j in range(n):
            if j < n:
                scan_x[j] = 0.0
        cuda.syncthreads()

        # Compute scan_x[j] = exp(delta * A[j]) * scan_x[j] + delta * B[j] * u
        for j in range(n):
            if j < n:
                scan_x[j] = math.exp(delta_val * A[dim, j]) * scan_x[j] + delta_val * B[batch, seq, j] * u_val
        cuda.syncthreads()

        # Store original scan_x before scan for inclusive scan
        for j in range(n):
            if j < n:
                input_x[j] = scan_x[j]
        cuda.syncthreads()

        # Perform Blelloch's exclusive scan on scan_x
        step = 1
        while step < n:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                scan_x[index] += scan_x[index - step]
            cuda.syncthreads()
            step *= 2

        if cuda.threadIdx.x == 0 and n > 0:
            scan_x[n - 1] = 0.0
        cuda.syncthreads()

        step = n // 2
        while step >= 1:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                t = scan_x[index - step]
                scan_x[index - step] = scan_x[index]
                scan_x[index] += t
            cuda.syncthreads()
            step //= 2

        # Inclusive scan: add input_x[j] to scan_x[j]
        for j in range(n):
            if j < n:
                scan_x[j] += input_x[j]
        cuda.syncthreads()

        # Now scan_x contains the inclusive scan result
        # Compute y = sum(scan_x[j] * C[j]) + u * D[dim]
        y = 0.0
        for j in range(n):
            if j < n:
                y += scan_x[j] * C[batch, seq, j]
        y += u_val * D[dim]

        output[batch, seq, dim] = y



    @cuda.jit
    def fused_scan_bwd_kernel(u, delta, A, B, C, D, grad_output, grad_u, grad_delta,
                              grad_A, grad_B, grad_C, grad_D, b, l, d_in, n):
        """
        Backward pass kernel implementing Blelloch's inclusive scan and reverse scan for gradient computations.
        """
        # Allocate shared memory: double the size for input and scan
        shared_mem = cuda.shared.array(shape=1024, dtype=float64)  # Adjust size if needed
        input_x = shared_mem[:256]
        scan_x = shared_mem[256:512]
        grad_shared_A = shared_mem[512:768]
        grad_shared_B = shared_mem[768:1024]

        # Initialize shared gradients to zero
        for j in range(n):
            if j < n:
                grad_shared_A[j] = 0.0
                grad_shared_B[j] = 0.0
        cuda.syncthreads()

        idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        total = b * l * d_in

        if idx >= total:
            return

        dim = idx % d_in
        seq = (idx // d_in) % l
        batch = idx // (l * d_in)

        u_val = u[batch, seq, dim]
        delta_val = delta[batch, seq, dim]

        y_grad = grad_output[batch, seq, dim]

        # Initialize scan_x to zero
        for j in range(n):
            if j < n:
                scan_x[j] = 0.0
        cuda.syncthreads()

        # Recompute scan_x as in forward pass
        for j in range(n):
            if j < n:
                scan_x[j] = math.exp(delta_val * A[dim, j]) * scan_x[j] + delta_val * B[batch, seq, j] * u_val
        cuda.syncthreads()

        # Store original scan_x before scan for inclusive scan
        for j in range(n):
            if j < n:
                input_x[j] = scan_x[j]
        cuda.syncthreads()

        # Perform Blelloch's exclusive scan on scan_x
        step = 1
        while step < n:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                scan_x[index] += scan_x[index - step]
            cuda.syncthreads()
            step *= 2

        if cuda.threadIdx.x == 0 and n > 0:
            scan_x[n - 1] = 0.0
        cuda.syncthreads()

        step = n // 2
        while step >= 1:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                t = scan_x[index - step]
                scan_x[index - step] = scan_x[index]
                scan_x[index] += t
            cuda.syncthreads()
            step //= 2

        # Inclusive scan: add input_x[j] to scan_x[j]
        for j in range(n):
            if j < n:
                scan_x[j] += input_x[j]
        cuda.syncthreads()

        # Reverse scan to propagate gradients

        # Initialize gradient accumulation for A and B
        for j in range(n):
            if j < n:
                grad_shared_A[j] = 0.0
                grad_shared_B[j] = 0.0
        cuda.syncthreads()

        # Compute gradients
        for j in range(n):
            if j < n:
                # Gradient w.r.t C[j]
                grad_C_val = scan_x[j] * y_grad
                grad_C[batch, seq, j] += grad_C_val

                # Gradient w.r.t A[j]
                grad_A_val = delta_val * math.exp(delta_val * A[dim, j]) * scan_x[j] * y_grad
                grad_shared_A[j] += grad_A_val

                # Gradient w.r.t B[j]
                grad_B_val = delta_val * u_val * y_grad
                grad_shared_B[j] += grad_B_val

                # Gradient w.r.t D[dim]
                grad_D[dim] += u_val * y_grad

        # Now, accumulate shared gradients for A and B
        for j in range(n):
            if j < n:
                A_grad = grad_shared_A[j]
                B_grad = grad_shared_B[j]
                grad_A[dim, j] += A_grad
                grad_B[batch, seq, j] += B_grad

        # Gradient w.r.t delta
        for j in range(n):
            if j < n:
                grad_delta_val = y_grad * (A[dim, j] * math.exp(delta_val * A[dim, j]) * scan_x[j] +
                                            B[batch, seq, j] * u_val)
                grad_delta[batch, seq, dim] += grad_delta_val

        # Gradient w.r.t u
        grad_u_val = 0.0
        for j in range(n):
            if j < n:
                grad_u_val += delta_val * B[batch, seq, j] * C[batch, seq, j] * y_grad
        grad_u[batch, seq, dim] += grad_u_val


    # -------------------------------
    # PyTorch Autograd Function
    # -------------------------------

    class FusedScanFunction(torch.autograd.Function):
        @staticmethod
        def forward(ctx, u, delta, A, B, C, D):
            """
            Forward pass using the custom CUDA kernel.
            """
            # Detach inputs to prevent PyTorch from tracking gradients through them
            u_detached = u.detach().contiguous().double().to(device)
            delta_detached = delta.detach().contiguous().double().to(device)
            A_detached = A.detach().contiguous().double().to(device)
            B_detached = B.detach().contiguous().double().to(device)
            C_detached = C.detach().contiguous().double().to(device)
            D_detached = D.detach().contiguous().double().to(device)

            b, l, d_in = u_detached.shape
            n = A_detached.shape[1]

            # Allocate output tensor
            output = torch.zeros_like(u_detached, device=device, dtype=torch.float64)

            # Define grid and block dimensions
            threads_per_block = 256
            blocks_per_grid = (b * l * d_in + (threads_per_block - 1)) // threads_per_block

            # Launch the forward kernel
            fused_scan_fwd_kernel[blocks_per_grid, threads_per_block](
                cuda.as_cuda_array(u_detached),
                cuda.as_cuda_array(delta_detached),
                cuda.as_cuda_array(A_detached),
                cuda.as_cuda_array(B_detached),
                cuda.as_cuda_array(C_detached),
                cuda.as_cuda_array(D_detached),
                cuda.as_cuda_array(output),
                b, l, d_in, n
            )

            # Save context for backward
            ctx.save_for_backward(u_detached, delta_detached, A_detached, B_detached, C_detached, D_detached)
            ctx.n = n
            ctx.b = b
            ctx.l = l
            ctx.d_in = d_in

            return output

        @staticmethod
        def backward(ctx, grad_output):
            u_detached, delta_detached, A_detached, B_detached, C_detached, D_detached = ctx.saved_tensors
            n = ctx.n
            b = ctx.b
            l = ctx.l
            d_in = ctx.d_in

            grad_output_detached = grad_output.contiguous().double().to(device)

            # Allocate gradient tensors
            grad_u = torch.zeros_like(u_detached, device=device, dtype=torch.float64)
            grad_delta = torch.zeros_like(delta_detached, device=device, dtype=torch.float64)
            grad_A = torch.zeros_like(A_detached, device=device, dtype=torch.float64)
            grad_B = torch.zeros_like(B_detached, device=device, dtype=torch.float64)
            grad_C = torch.zeros_like(C_detached, device=device, dtype=torch.float64)
            grad_D = torch.zeros_like(D_detached, device=device, dtype=torch.float64)

            # Define grid and block dimensions
            threads_per_block = 256
            blocks_per_grid = (b * l * d_in + (threads_per_block - 1)) // threads_per_block

            # Launch the backward kernel
            fused_scan_bwd_kernel[blocks_per_grid, threads_per_block](
                cuda.as_cuda_array(u_detached),
                cuda.as_cuda_array(delta_detached),
                cuda.as_cuda_array(A_detached),
                cuda.as_cuda_array(B_detached),
                cuda.as_cuda_array(C_detached),
                cuda.as_cuda_array(D_detached),
                cuda.as_cuda_array(grad_output_detached),
                cuda.as_cuda_array(grad_u),
                cuda.as_cuda_array(grad_delta),
                cuda.as_cuda_array(grad_A),
                cuda.as_cuda_array(grad_B),
                cuda.as_cuda_array(grad_C),
                cuda.as_cuda_array(grad_D),
                b, l, d_in, n
            )

            return grad_u, grad_delta, grad_A, grad_B, grad_C, grad_D


    # -------------------------------
    # Custom PyTorch Module
    # -------------------------------


    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F
    from torch.autograd import Function
    from torch.utils.checkpoint import checkpoint
    import numpy as np
    from numba import cuda
    import math
    from einops import repeat
    import sys
    from torch.optim.lr_scheduler import CosineAnnealingLR
    import os
    import math
    import random
    import shutil
    import re
    from collections import Counter

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.cpp_extension import load
    from torch.utils.checkpoint import checkpoint
    from safetensors.torch import save_model, load_model
    from torch.nn.utils.rnn import pad_sequence
    from torch.optim import Optimizer
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from torch.autograd import Function
    from einops import repeat
    from transformers import GPT2Tokenizer
    from datasets import load_dataset
    from tqdm import tqdm

    # ============================
    # CUDA Kernels
    # ============================

    # Wrapper function to launch the forward kernel
    def fused_scan_fwd(u, delta, A, B, C, D):
        b, l, d_in = u.shape
        n = A.shape[1]
        output = cuda.device_array_like(u)

        threads_per_block = 256
        blocks_per_grid = (b * l * d_in + threads_per_block - 1) // threads_per_block
        blocks_per_grid = max(blocks_per_grid, 2)  # You can adjust this based on performance
        # shared_mem_size is not needed since we're using fixed shared memory

        fused_scan_fwd_kernel[blocks_per_grid, threads_per_block](
            u, delta, A, B, C, D, output, b, l, d_in, n
        )
        return output.copy_to_host()

    # Wrapper function to launch the backward kernel
    def fused_scan_bwd(u, delta, A, B, C, D, grad_output):
        b, l, d_in = u.shape
        n = A.shape[1]
        grad_u = cuda.device_array_like(u)
        grad_delta = cuda.device_array_like(delta)
        grad_A = cuda.device_array_like(A)
        grad_B = cuda.device_array_like(B)
        grad_C = cuda.device_array_like(C)
        grad_D = cuda.device_array_like(D)

        threads_per_block = 256
        blocks_per_grid = (b * l * d_in + threads_per_block - 1) // threads_per_block
        blocks_per_grid = max(blocks_per_grid, 2)  # You can adjust this based on performance
        # shared_mem_size is not needed since we're using fixed shared memory

        fused_scan_bwd_kernel[blocks_per_grid, threads_per_block](
            u, delta, A, B, C, D, grad_output, grad_u, grad_delta, grad_A, grad_B, grad_C, grad_D,
            b, l, d_in, n
        )
        return grad_u.copy_to_host(), grad_delta.copy_to_host(), grad_A.copy_to_host(), grad_B.copy_to_host(), grad_C.copy_to_host(), grad_D.copy_to_host()

    # ============================
    # Custom Layers and Functions
    # ============================

    def compute_dropout_rate(batch_idx, total_batches, max_dropout=0.4):
        """Compute dropout rate linearly increasing from 0 to max_dropout."""
        return min(max_dropout * (batch_idx / total_batches), max_dropout)


    class RMSNorm(nn.Module):
        def __init__(self, d_model: int, eps: float = 1e-5, use_mup: bool = False):
            super().__init__()

            self.use_mup = use_mup
            self.eps = eps

            # RMSNorm gains prevent muTransfer (section 4.2.3)
            if not use_mup:
                self.weight = nn.Parameter(torch.ones(d_model))

        def forward(self, x):
            output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

            if not self.use_mup:
                return output * self.weight
            else:
                return output


    class SelectiveScanFunction(Function):
        @staticmethod
        def forward(ctx, u, delta, A_log, B, C, D):
            A = -torch.exp(A_log.float())
            # Ensure all tensors are contiguous and on the correct device
            u_contig = u.contiguous()
            delta_contig = delta.contiguous()
            A_contig = A.contiguous()
            B_contig = B.contiguous()
            C_contig = C.contiguous()
            D_contig = D.contiguous()

            # Move to Numba-compatible device arrays
            u_device = cuda.to_device(u_contig.cpu().numpy())
            delta_device = cuda.to_device(delta_contig.cpu().numpy())
            A_device = cuda.to_device(A_contig.cpu().numpy())
            B_device = cuda.to_device(B_contig.cpu().numpy())
            C_device = cuda.to_device(C_contig.cpu().numpy())
            D_device = cuda.to_device(D_contig.cpu().numpy())

            # Launch the forward kernel
            output_host = fused_scan_fwd(u_device, delta_device, A_device, B_device, C_device, D_device)

            # Convert output back to torch tensor
            output = torch.from_numpy(output_host).to(u.device)

            ctx.save_for_backward(u, delta, A_log, B, C, D)
            return output

        @staticmethod
        def backward(ctx, grad_output):
            u, delta, A_log, B, C, D = ctx.saved_tensors
            A = -torch.exp(A_log)

            # Move tensors to CPU and convert to NumPy
            u_cpu = u.contiguous().cpu().numpy()
            delta_cpu = delta.contiguous().cpu().numpy()
            A_cpu = A.contiguous().cpu().numpy()
            B_cpu = B.contiguous().cpu().numpy()
            C_cpu = C.contiguous().cpu().numpy()
            D_cpu = D.contiguous().cpu().numpy()
            grad_output_cpu = grad_output.contiguous().cpu().numpy()

            # Move to Numba-compatible device arrays
            u_device = cuda.to_device(u_cpu)
            delta_device = cuda.to_device(delta_cpu)
            A_device = cuda.to_device(A_cpu)
            B_device = cuda.to_device(B_cpu)
            C_device = cuda.to_device(C_cpu)
            D_device = cuda.to_device(D_cpu)
            grad_output_device = cuda.to_device(grad_output_cpu)

            # Launch the backward kernel
            grad_u_host, grad_delta_host, grad_A_host, grad_B_host, grad_C_host, grad_D_host = fused_scan_bwd(
                u_device, delta_device, A_device, B_device, C_device, D_device, grad_output_device
            )

            # Convert gradients back to torch tensors
            grad_u = torch.from_numpy(grad_u_host).to(u.device)
            grad_delta = torch.from_numpy(grad_delta_host).to(delta.device)
            grad_A = torch.from_numpy(grad_A_host).to(A.device)
            grad_B = torch.from_numpy(grad_B_host).to(B.device)
            grad_C = torch.from_numpy(grad_C_host).to(C.device)
            grad_D = torch.from_numpy(grad_D_host).to(D.device)

            # Compute grad_A_log
            grad_A_log = grad_A * (-torch.exp(A_log))

            return grad_u, grad_delta, grad_A_log, grad_B, grad_C, grad_D


    class SelectiveScanModel(nn.Module):
        def __init__(self, d_in, d_state, dt_rank):
            super().__init__()
            self.norm = RMSNorm(d_in)
            self.d_state = d_state
            self.dt_rank = dt_rank
            A = repeat(
                torch.arange(1, self.d_state + 1, dtype=torch.float32),
                "n -> d n",
                d=d_in,
            ).contiguous()
            A_log = torch.log(A)
            self.A_log = nn.Parameter(A_log)
            self.D = nn.Parameter(torch.ones(d_in).cuda())
            self.x_proj = nn.Linear(d_in, dt_rank + d_state * 2, bias=True).cuda()
            self.dt_proj = nn.Linear(dt_rank, d_in, bias=True).cuda()
            self.output_layer = nn.Linear(d_in, d_in).cuda()

        def forward(self, x):
            b, l, d_in = x.shape
            D = self.D
            x_dbl = self.x_proj(x)
            delta, B, C = x_dbl.split([self.dt_rank, self.d_state, self.d_state], dim=-1)
            delta = F.softplus(self.dt_proj(delta))
            y = self.norm(SelectiveScanFunction.apply(x, delta, self.A_log, B, C, D))
            return self.output_layer(y)


    class MambaBlock(nn.Module):
        def __init__(self, d_model, d_state=16, dt_rank=16, d_conv=4, expand=2, conv_bias=True, bias=False):
            super().__init__()
            self.d_model = d_model
            self.d_state = d_state
            self.d_conv = d_conv
            self.expand = expand
            self.d_inner = int(self.expand * self.d_model)
            self.dt_rank = dt_rank
            self.in_proj = nn.Linear(self.d_model, self.d_inner * 2, bias=bias)
            self.conv1d = nn.Sequential(
                nn.Conv1d(in_channels=self.d_inner, out_channels=self.d_inner, kernel_size=d_conv, groups=self.d_inner, bias=conv_bias),
                nn.Conv1d(in_channels=self.d_inner, out_channels=self.d_inner, kernel_size=1, bias=conv_bias)
            )
            self.activation = nn.SiLU()
            self.ssm_model = SelectiveScanModel(self.d_inner, self.d_state, self.dt_rank)
            self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias)
            self.norm = RMSNorm(self.d_model)
        def forward(self, x):
            b, l, d = x.shape
            x_proj = self.in_proj(x)
            x, res = x_proj.chunk(2, dim=-1)
            x = x.transpose(1, 2)
            x = F.pad(x, (self.d_conv - 1, 0))
            x = self.conv1d(x)[:, :, :l]
            x = x.transpose(1, 2)
            x = self.activation(x)
            y = self.ssm_model(x)
            y = y * self.activation(res)
            y = self.norm(self.out_proj(y))
            return y


    class BlockStack(nn.Module):
        def __init__(self, num_blocks, d_model, d_state, dt_rank):
            super().__init__()
            self.blocks = nn.ModuleList([MambaBlock(d_model, d_state, dt_rank) for _ in range(num_blocks)])

        def forward(self, x, dropout_rate=None):
            x_res = x
            for i in range(len(self.blocks)):
                x = self.blocks[i](x)
            return x * x_res


    class FeedForwardLayer(nn.Module):
        def __init__(self, embed_dim, d_model, hidden_dim, window_size=4):
            """
            Initializes the FeedForwardLayer with windowed masking.

            Args:
                d_model (int): Dimension of the input and output features.
                hidden_dim (int): Dimension of the hidden layer.
                window_size (int): Size of the window around each position.
            """
            super().__init__()
            self.fc1 = nn.Linear(embed_dim, hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, d_model)
            self.activation = nn.SiLU()
            self.norm = RMSNorm(d_model)

        def forward(self, x):
            """
            Forward pass of the FeedForwardLayer with windowed masking.

            Args:
                x (torch.Tensor): Input tensor of shape (batch_size, seq_length, d_model).

            Returns:
                torch.Tensor: Output tensor of shape (batch_size, seq_length, d_model).
            """

            # Manually apply the windowed mask to the weights of the first linear layer
            masked_weight = self.fc1.weight  # Element-wise multiplication
            masked_bias = self.fc1.bias

            # Perform the masked linear transformation
            x = torch.matmul(x, masked_weight.t()) + masked_bias

            x = self.activation(x)
            x = self.fc2(x)
            x = self.norm(x)
            return x
    class HeadModel(nn.Module):
        def __init__(self, vocab_size, embed_dim, d_state, dt_rank, d_model, num_blocks=1, num_matrices=2):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embed_dim)
            self.block_stacks = nn.ModuleList([
                BlockStack(num_blocks, embed_dim // num_matrices, d_state, dt_rank)
                for _ in range(num_matrices)
            ])
            self.activation = nn.SiLU()
            self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
            self.policy_head = nn.Linear(d_model, 1, bias=False)  # New head for policy output (scalar for each token)
            self.norm = nn.LayerNorm(d_model)
            hidden_dim = d_model * 2
            self.feed_forward = FeedForwardLayer(embed_dim, d_model, hidden_dim)
            self.num_matrices = num_matrices
    
        def forward(self, input_ids, dropout_rate=None):
            x = self.embedding(input_ids)
            e = x
            split_dim = x.shape[2] // self.num_matrices
            x_splits = torch.split(x, split_dim, dim=2)
            processed_splits = []
            for i, feature_matrix in enumerate(x_splits):
                block_stack = self.block_stacks[i]
                processed_matrix = checkpoint(block_stack, feature_matrix, use_reentrant=False)
                processed_splits.append(processed_matrix)
            x = torch.cat(processed_splits, dim=2)
            x = self.feed_forward(x + e)
            x = self.norm(x)
            
            logits = self.lm_head(x)  # Output for language modeling
            policy_values = self.policy_head(x).squeeze(-1)  # Output policy values (scalar for each token)
            
            return logits, policy_values



    # ============================
    # 3. Data Preparation
    # ============================

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})
    dataset = load_dataset("togethercomputer/RedPajama-Data-V2", "default", languages=["en"], split="train", streaming=True, trust_remote_code=True)
    import torch
    import textstat
    
    def compute_fk_reward(decoded_texts):
        """
        Compute Flesch-Kincaid readability scores as rewards for a batch of decoded texts.
    
        Args:
            decoded_texts (list of str): List of generated texts.
    
        Returns:
            torch.Tensor: FK readability scores normalized to be positive rewards using soft clamping.
        """
        rewards = []
        for text in decoded_texts:
            fk_score = textstat.flesch_kincaid_grade(text)
    
            # Convert fk_score and raw_reward to a tensor
            raw_reward = torch.tensor(fk_score/100, dtype=torch.float32).cuda()
    
            # Apply a sigmoid-based function to softly clamp between 0 and 10
            reward = 10 / (1 + torch.exp(-raw_reward / 2))  # Scaling to control the smoothness
    
            rewards.append(raw_reward)
    
        # Convert the list of rewards to a torch tensor for batch processing
        rewards = torch.stack(rewards)  # Stack the list into a single tensor
        return rewards



    def collate_fn(batch, tokenizer):
        tokenized_inputs = [item['tokenized_input'] for item in batch]
        tokenized_targets = [item['tokenized_target'] for item in batch]
        bos_token_id = tokenizer.bos_token_id
        eos_token_id = tokenizer.eos_token_id
        tokenized_inputs = [
            torch.cat([torch.tensor([bos_token_id]), tokenized_input, torch.tensor([eos_token_id])])
            for tokenized_input in tokenized_inputs
        ]
        tokenized_targets = [
            torch.cat([torch.tensor([bos_token_id]), tokenized_target, torch.tensor([eos_token_id])])
            for tokenized_target in tokenized_targets
        ]
        padded_inputs = pad_sequence(tokenized_inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
        padded_targets = pad_sequence(tokenized_targets, batch_first=True, padding_value=tokenizer.pad_token_id)
        return padded_inputs, padded_targets


    class BufferedStreamingDataLoader:
        def __init__(self, dataset, batch_size, buffer_size, collate_fn, tokenizer, shuffle_prob=0.3, mask_prob=0.5):
            self.dataset = dataset
            self.batch_size = batch_size
            self.buffer_size = buffer_size
            self.collate_fn = collate_fn
            self.tokenizer = tokenizer
            self.shuffle_prob = shuffle_prob
            self.mask_prob = mask_prob

        def _split_into_sentences(self, tokenized_input):
            sentence_end_tokens = {
                self.tokenizer.convert_tokens_to_ids('.'),
                self.tokenizer.convert_tokens_to_ids('!'),
                self.tokenizer.convert_tokens_to_ids('?')
            }
            sentences = []
            current_sentence = []
            for token in tokenized_input:
                current_sentence.append(token)
                if token in sentence_end_tokens:
                    sentences.append(current_sentence)
                    current_sentence = []
            if current_sentence:
                sentences.append(current_sentence)
            return sentences

        def _shuffle_sentences(self, tokenized_input):
            sentences = self._split_into_sentences(tokenized_input)
            random.shuffle(sentences)
            shuffled_input = [token for sentence in sentences for token in sentence]
            return shuffled_input

        def _apply_random_mask(self, tokenized_input):
            length = len(tokenized_input)
            if length > 1:
                mask_start = int(length * 0.33)
                mask_end = int(length * 0.66)
                random_mask_start = random.randint(mask_start, mask_end)
                mask_length = random.randint(int(0.33 * length), int(0.66 * length))
                mask_end_idx = length
                masked_input = tokenized_input.clone()
                mask_token_id = self.tokenizer.pad_token_id
                masked_input[random_mask_start:mask_end_idx] = mask_token_id
                return masked_input
            return tokenized_input

        def __iter__(self):
            buffer = []
            token_counter = Counter()
            for item in self.dataset:
                tokenized_input = self.tokenizer(
                    item['raw_content'], return_tensors='pt', max_length=4096, padding=False, truncation=True
                )['input_ids'][0]
                tokenized_target = self.tokenizer(
                    item['raw_content'], return_tensors='pt', max_length=4096, padding=False, truncation=True
                )['input_ids'][0]

                if random.random() < self.shuffle_prob:
                    shuffled_input = self._shuffle_sentences(tokenized_input.tolist())
                    tokenized_input = torch.tensor(shuffled_input)
                else:
                    tokenized_input = tokenized_input
                    if random.random() < self.mask_prob:
                        tokenized_input = self._apply_random_mask(tokenized_input)

                buffer.append({
                    'text':  item['raw_content'],
                    'tokenized_input': tokenized_input,
                    'tokenized_target': tokenized_target
                })
                token_counter.update(tokenized_input.tolist())

                if len(buffer) == self.buffer_size:
                    buffer = sorted(buffer, key=lambda x: len(x['tokenized_input']), reverse=False)
                    for i in range(0, len(buffer), self.batch_size):
                        batch = buffer[i:i + self.batch_size]
                        yield self.collate_fn(batch, self.tokenizer), token_counter
                    buffer = []
                    token_counter = Counter()

            if buffer:
                buffer = sorted(buffer, key=lambda x: len(x['tokenized_input']), reverse=True)
                for i in range(0, len(buffer), self.batch_size):
                    batch = buffer[i:i + self.batch_size]
                    yield self.collate_fn(batch, self.tokenizer), token_counter


    # ============================
    # 4. Model Initialization
    # ============================

    def initialize_model_parameters(model):
        """ Initialize all trainable parameters in the model. """
        for name, param in model.named_parameters():
            if 'weight' in name:
                if len(param.shape) > 1:
                    nn.init.xavier_uniform_(param)
                else:
                    nn.init.normal_(param, 0.0, 0.02)
            elif 'bias' in name:
                nn.init.zeros_(param)


    batch_size = 1
    buffer_size = 100 * batch_size
    dataloader = BufferedStreamingDataLoader(
        dataset, batch_size=batch_size, buffer_size=buffer_size, collate_fn=collate_fn, tokenizer=tokenizer
    )

    vocab_size = tokenizer.vocab_size + 3
    embed_dim = 512
    d_state = 16
    d_model = 1024
    dt_rank = math.ceil(d_model / 16)
    model = HeadModel(vocab_size, embed_dim, d_state, dt_rank, d_model).cuda()
    initialize_model_parameters(model)
    model = nn.DataParallel(model)


    # ============================
    # 5. Optimizer and Scheduler
    # ============================

    import math
    import torch
    from torch.optim import Optimizer
    import math
    import torch
    from torch.optim import Optimizer
    from torch.autograd import Variable

    class AdEMAMix(Optimizer):
        def __init__(self, params, lr=1e-3, betas=(0.95, 0.979, 0.9995), eps=1e-7,
                      weight_decay=0.001, alpha=10.0, T_alpha_beta3=300000, max_steps=300000,
                      beta_opposite=0.99, nystrom_rank=200, nystrom_interval=50):
            if not 0.0 <= lr:
                raise ValueError(f"Invalid learning rate: {lr}")
            if not 0.0 <= eps:
                raise ValueError(f"Invalid epsilon value: {eps}")
            assert len(betas) == 3, f"Invalid beta parameters: {betas}, expected 3"
            assert all(0.0 <= beta < 1.0 for beta in betas), f"Invalid beta parameters: {betas}"
            if not 0.0 <= weight_decay:
                raise ValueError(f"Invalid weight_decay value: {weight_decay}")

            defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                            alpha=alpha, T_alpha_beta3=T_alpha_beta3)
            super(AdEMAMix, self).__init__(params, defaults)

            self.global_step = 0
            self.beta1, self.beta2, self.beta3 = betas
            self.alpha = alpha
            self.T_alpha_beta3 = T_alpha_beta3
            self.max_steps = max_steps
            self.beta_opposite = beta_opposite

            # Nyström parameters
            self.nystrom_rank = nystrom_rank
            self.nystrom_interval = nystrom_interval
            self.gradient_samples = []  # Buffer to store gradient samples
            self.nystrom_matrix = None  # Approximated covariance matrix

            # Precompute beta powers
            self.beta1_pows = [1.0]
            self.beta2_pows = [1.0]
            self.beta3_pows = [1.0]
            self.beta_opposite_pows = [1.0]
            for step in range(1, max_steps + 1):
                self.beta1_pows.append(self.beta1_pows[-1] * self.beta1)
                self.beta2_pows.append(self.beta2_pows[-1] * self.beta2)
                self.beta_opposite_pows.append(self.beta_opposite_pows[-1] * self.beta_opposite)
                if self.T_alpha_beta3 is not None:
                    alpha_t = min(step * self.alpha / self.T_alpha_beta3, self.alpha)
                    exponent = (math.log(self.beta1) * math.log(self.beta3)) / (
                        (1 - step / self.T_alpha_beta3) * math.log(self.beta3) +
                        (step / self.T_alpha_beta3) * math.log(self.beta1)
                    )
                    beta3_t = min(math.exp(exponent), self.beta3)
                else:
                    beta3_t = self.beta3
                self.beta3_pows.append(beta3_t)

        def __setstate__(self, state):
            super(AdEMAMix, self).__setstate__(state)

        @torch.no_grad()
        def step(self, closure=None):
            loss = None
            if closure is not None:
                with torch.enable_grad():
                    loss = closure()

            self.global_step += 1  # Increment global step once per step

            for group in self.param_groups:
                params_with_grad = []
                grads = []
                exp_avgs = []
                exp_avg_sqs = []
                exp_avg_slow = []
                exp_avg_opposite = []

                for p in group['params']:
                    if p.grad is not None:
                        if p.grad.is_sparse:
                            raise RuntimeError('AdEMAMixNyström does not support sparse gradients')
                        params_with_grad.append(p)
                        grads.append(p.grad)

                        state = self.state[p]
                        if len(state) == 0:
                            state['step'] = 0
                            state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_slow'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_opposite'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                        exp_avgs.append(state['exp_avg'])
                        exp_avg_sqs.append(state['exp_avg_sq'])
                        exp_avg_slow.append(state['exp_avg_slow'])
                        exp_avg_opposite.append(state['exp_avg_opposite'])
                        state['step'] += 1

                beta1, beta2, beta3 = group['betas']
                alpha = group['alpha']
                T_alpha_beta3 = group['T_alpha_beta3']

                step = self.global_step
                if step <= self.max_steps:
                    bias_correction1 = 1 - self.beta1_pows[step]
                    bias_correction2 = 1 - self.beta2_pows[step]
                    bias_correction_opposite = 1 - self.beta_opposite_pows[step]
                    if T_alpha_beta3 is not None:
                        beta3_t = self.beta3_pows[step]
                    else:
                        beta3_t = beta3
                else:
                    bias_correction1 = 1 - self.beta1 ** step
                    bias_correction2 = 1 - self.beta2 ** step
                    bias_correction_opposite = 1 - self.beta_opposite ** step
                    if T_alpha_beta3 is not None:
                        alpha_t = min(step * alpha / T_alpha_beta3, alpha)
                        beta3_t = min(math.exp(math.log(beta1) * math.log(beta3) / 
                                      ((1 - step / T_alpha_beta3) * math.log(beta3) + 
                                        (step / T_alpha_beta3) * math.log(beta1))), self.beta3)
                    else:
                        beta3_t = beta3

                # Update Nyström samples
                if step % self.nystrom_interval == 0:
                    self._update_nystrom(params_with_grad, grads)

                # Compute preconditioning matrix using Nyström approximation
                if self.nystrom_matrix is not None:
                    precondition_matrix = self._compute_preconditioning()
                else:
                    precondition_matrix = None

                # Update parameters
                for i, param in enumerate(params_with_grad):
                    grad = grads[i]
                    exp_avg = exp_avgs[i]
                    exp_avg_sq = exp_avg_sqs[i]
                    exp_avg_slow_i = exp_avg_slow[i]
                    exp_avg_opposite_i = exp_avg_opposite[i]

                    # Update biased first moment estimate
                    exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)

                    # Update opposing EMA
                    exp_avg_opposite_i.mul_(self.beta_opposite).add_(grad.neg(), alpha=1 - self.beta_opposite)

                    # Update biased second moment estimate
                    exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
                    # Update slow EMA
                    exp_avg_slow_i.mul_(beta3_t).add_(grad, alpha=1 - beta3_t)

                    # Combine primary and opposing EMAs
                    combined_momentum = exp_avg + alpha * exp_avg_slow_i + exp_avg_opposite_i

                    denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                    step_size = group['lr'] / bias_correction1

                    if group['weight_decay'] != 0:
                        param.add_(param, alpha=-group['weight_decay'] * group['lr'])

                    # Apply preconditioning if available
                    if precondition_matrix is not None:
                        combined_momentum = precondition_matrix.matmul(combined_momentum.view(-1, 1)).view_as(combined_momentum)

                    param.addcdiv_(combined_momentum, denom, value=-step_size)

            return loss

        def _update_nystrom(self, params, grads):
            """
            Update the gradient samples used for Nyström approximation.
            """
            # Flatten and concatenate all gradients into a single vector
            flat_grads = torch.cat([g.view(-1) for g in grads])
            flat_grads = flat_grads.cpu().detach()

            # Append to samples
            self.gradient_samples.append(flat_grads)

            # Maintain only the latest nystrom_rank samples
            if len(self.gradient_samples) > self.nystrom_rank:
                self.gradient_samples.pop(0)

            # Construct the Nyström matrix if enough samples are available
            if len(self.gradient_samples) == self.nystrom_rank:
                # Create a matrix where each column is a sampled gradient
                G = torch.stack(self.gradient_samples, dim=1)  # Shape: (num_params, nystrom_rank)
                # Compute the covariance matrix
                C = G @ G.t() / self.nystrom_rank  # Shape: (num_params, num_params)
                # Perform eigendecomposition
                eigenvalues, eigenvectors = torch.linalg.eigh(C)
                # Select the top k eigenvalues and eigenvectors
                idx = torch.argsort(eigenvalues, descending=True)[:self.nystrom_rank]
                self.nystrom_matrix = eigenvectors[:, idx] @ torch.diag(1.0 / torch.sqrt(eigenvalues[idx] + 1e-10)) @ eigenvectors[:, idx].t()

        def _compute_preconditioning(self):
            """
            Compute the preconditioning matrix using the Nyström approximation.
            """
            return self.nystrom_matrix.to(next(self.param_groups[0]['params']).device)





    optimizer = AdEMAMix(model.parameters(), lr=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=500000, eta_min=1e-6)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    # ============================
    # 6. Checkpointing Functions
    # ============================

    checkpoint_dir = "/checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    max_checkpoints = 5  # Maximum number of checkpoints to keep

    def save_model_checkpoint(model, epoch, batch_idx):
        model_dir = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}")
        os.makedirs(model_dir, exist_ok=True)

        # Save the entire model using safetensors to handle shared tensors
        model_to_save = model.module if isinstance(model, nn.DataParallel) else model
        save_model(model_to_save, os.path.join(model_dir, "model.safetensors"))

        cleanup_old_checkpoints()

    def save_optimizer_checkpoint(optimizer, epoch, batch_idx):
        optimizer_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "optimizer.pt")
        torch.save(optimizer.state_dict(), optimizer_state_file)

    def save_scheduler_checkpoint(scheduler, epoch, batch_idx):
        scheduler_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "scheduler.pt")
        torch.save(scheduler.state_dict(), scheduler_state_file)

    def save_training_state(epoch, batch_idx, loss_history):
        training_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "training_state.pt")
        training_state = {
            'epoch': epoch,
            'batch_idx': batch_idx,
            'loss_history': loss_history
        }
        torch.save(training_state, training_state_file)

    def cleanup_old_checkpoints():
        checkpoints = sorted(
            [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))],
            key=lambda x: os.path.getctime(os.path.join(checkpoint_dir, x)),
            reverse=True
        )
        if len(checkpoints) > max_checkpoints:
            old_checkpoints = checkpoints[max_checkpoints:]
            for old_checkpoint in old_checkpoints:
                old_checkpoint_path = os.path.join(checkpoint_dir, old_checkpoint)
                shutil.rmtree(old_checkpoint_path)
    import os
    import re
    import torch
    from torch import nn
    from safetensors.torch import load_file  # Import the correct function to load state_dict
    from safetensors.torch import load_file  # Ensure this import is present

    def load_latest_checkpoint(model, optimizer, scheduler, checkpoint_dir="/checkpoints"):
        """
        Load the second from the last checkpoint from the specified checkpoint directory.

        Parameters:
        - model (torch.nn.Module): The model to load the state into.
        - optimizer (torch.optim.Optimizer): The optimizer to load the state into.
        - scheduler (torch.optim.lr_scheduler._LRScheduler): The scheduler to load the state into.
        - checkpoint_dir (str): Directory where checkpoints are stored.

        Returns:
        - epoch (int): The epoch number to resume from.
        - batch_idx (int): The batch index to resume from.
        - loss_history (list): History of loss values.
        - optimizer (torch.optim.Optimizer): Optimizer with loaded state.
        - scheduler (torch.optim.lr_scheduler._LRScheduler): Scheduler with loaded state.
        """

        # Ensure the checkpoint directory exists
        if not os.path.isdir(checkpoint_dir):
            print(f"Checkpoint directory '{checkpoint_dir}' does not exist. Starting training from scratch.")
            return 0, 0, [], optimizer, scheduler

        # List all checkpoint directories
        checkpoints = sorted(
            [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))],
            key=lambda x: os.path.getctime(os.path.join(checkpoint_dir, x)),
            reverse=True
        )

        # Check if there are at least two checkpoints
        if len(checkpoints) < 2:
            if len(checkpoints) == 1:
                print("Only one checkpoint found. Loading the latest checkpoint.")
                latest_checkpoint = checkpoints[0]
            else:
                print("No checkpoints found. Starting training from scratch.")
                return 0, 0, [], optimizer, scheduler
        else:
            # Load the second from the last checkpoint
            latest_checkpoint = checkpoints[1]
            print(f"Resuming training from second latest checkpoint: {latest_checkpoint}")

        # Extract epoch and batch index from the checkpoint name
        match = re.match(r"checkpoint_epoch_(\d+)_batch_(\d+)", latest_checkpoint)
        if match:
            epoch = int(match.group(1))
            batch_idx = int(match.group(2))
        else:
            print("Checkpoint naming convention not recognized. Starting from scratch.")
            return 0, 0, [], optimizer, scheduler

        checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)

        # Load the model state using safetensors
        model_to_load = model.module if isinstance(model, nn.DataParallel) else model
        model_state_path = os.path.join(checkpoint_path, "model.safetensors")
        if os.path.exists(model_state_path):
            load_model(model_to_load, model_state_path)
            print(f"Model state loaded from {model_state_path}.")
        else:
            print(f"Model state file '{model_state_path}' not found. Starting from scratch.")
            return 0, 0, [], optimizer, scheduler

        # Load optimizer state
        optimizer_state_file = os.path.join(checkpoint_path, "optimizer.pt")
        if os.path.exists(optimizer_state_file):
            optimizer.load_state_dict(torch.load(optimizer_state_file))
            print(f"Optimizer state loaded from {optimizer_state_file}.")
        else:
            print(f"Optimizer state file '{optimizer_state_file}' not found. Continuing without loading optimizer state.")

        # Load scheduler state
        scheduler_state_file = os.path.join(checkpoint_path, "scheduler.pt")
        if os.path.exists(scheduler_state_file):
            scheduler.load_state_dict(torch.load(scheduler_state_file))
            print(f"Scheduler state loaded from {scheduler_state_file}.")
        else:
            print(f"Scheduler state file '{scheduler_state_file}' not found. Continuing without loading scheduler state.")

        # Load training state (e.g., loss history)
        training_state_file = os.path.join(checkpoint_path, "training_state.pt")
        if os.path.exists(training_state_file):
            training_state = torch.load(training_state_file)
            loss_history = training_state.get('loss_history', [])
            print(f"Training state loaded from {training_state_file}.")
        else:
            loss_history = []
            print(f"Training state file '{training_state_file}' not found. Continuing without loading training state.")

        return 0, batch_idx, loss_history, optimizer, scheduler


    # ============================
    # 7. Training Loop
    # ============================

    def freeze_all_blocks(model):
        if isinstance(model, torch.nn.DataParallel):
            block_stacks = model.module.block_stacks
        else:
            block_stacks = model.block_stacks

        # Freeze all blocks and corresponding mlps in each block stack
        for block_stack in block_stacks:
            for idx, block in enumerate(block_stack.blocks):
                # Freeze all parameters in the blocks
                for param in block.parameters():
                    param.requires_grad = False


    def unfreeze_block_pair(model, block_indices):
        if isinstance(model, torch.nn.DataParallel):
            block_stacks = model.module.block_stacks
        else:
            block_stacks = model.block_stacks

        # Unfreeze only the specified blocks and corresponding mlps in each block stack
        for block_stack in block_stacks:
            for idx in block_indices:
                # Unfreeze the block
                for param in block_stack.blocks[idx].parameters():
                    param.requires_grad = True



    class BlockPairManager:
        def __init__(self, num_blocks, pair_size=2):
            self.num_blocks = num_blocks
            self.pair_size = pair_size
            self.block_pairs = self.generate_block_pairs()
            self.current_pair_idx = 0

        def generate_block_pairs(self):
            block_pairs = []
            for i in range(0, self.num_blocks, self.pair_size):
                pair = tuple(range(i, min(i + self.pair_size, self.num_blocks)))
                block_pairs.append(pair)
            return block_pairs

        def get_current_pair(self):
            return self.block_pairs[self.current_pair_idx]

        def rotate_block_pair(self):
            self.current_pair_idx = (self.current_pair_idx + 1) % len(self.block_pairs)

        def reset(self):
            self.current_pair_idx = 0

    # Define accumulation steps
    accumulation_steps = 8  # Adjust based on your memory constraints
    # Initialize EMA of rewards (can be zero initially or set to the first batch's rewards)
    ema = 0.0  # Start at zero or initialize with the first reward batch
    beta = 0.99  # Smoothing factor for EMA

    num_epochs = 3
    checkpoint_interval = 200
    rotation_interval = 500
    total_batches = 0
    loss_history = []
    num_blocks = 1
    block_pair_manager = BlockPairManager(num_blocks=num_blocks, pair_size=1)
    start_epoch, start_batch_idx, loss_history, optimizer, scheduler = load_latest_checkpoint(model, optimizer, scheduler)
    max_sampling_p = 0.9
    # Gumbel-Softmax Sampling
    def sample_gumbel_softmax(logits, temperature=1.0, hard=False):
        """
        Performs Gumbel-Softmax sampling on the logits.
        
        Args:
            logits (torch.Tensor): Logits to sample from.
            temperature (float): Temperature parameter for Gumbel-Softmax.
            hard (bool): Whether to return hard (one-hot) samples or soft samples.
            
        Returns:
            torch.Tensor: Sampled tokens in the relaxed form, or one-hot encoded if hard=True.
        """
        # Sample from the Gumbel distribution
        gumbel_noise = -torch.log(-torch.log(torch.rand_like(logits) + 1e-10) + 1e-10)
        
        # Add Gumbel noise to the logits and apply softmax with temperature
        y_soft = F.softmax((logits + gumbel_noise) / temperature, dim=-1)
        
        if hard:
            # Straight-through trick to obtain discrete tokens
            y_hard = torch.argmax(y_soft, dim=-1)
            y_hard_one_hot = F.one_hot(y_hard, num_classes=logits.size(-1)).float()
            y = (y_hard_one_hot - y_soft).detach() + y_soft  # Differentiable discrete tokens
        else:
            y = y_soft
    
        return y
    
    # Inside your training loop
    # Inside your training loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_loss = 0
        total_fk_ease = 0
        num_batches = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}")
        freeze_all_blocks(model)
        unfreeze_block_pair(model, block_pair_manager.get_current_pair())
    
        accumulation_counter = 0
        accumulated_loss = 0.0
        optimizer.zero_grad()
    
        for batch_idx, ((input_ids, labels), freq) in enumerate(progress_bar):
            if epoch == start_epoch and batch_idx < start_batch_idx:
                continue
    
            input_ids = input_ids.cuda()
            input_ids = input_ids[:, :-1]
            labels = labels[:, 1:].contiguous().view(-1).cuda()
            batch_size, seq_length = input_ids.size()
            p = min(batch_idx / 18000.0 * max_sampling_p, max_sampling_p)
            dropout_rate = compute_dropout_rate(batch_idx + total_batches * epoch, 30000)
    
            # Forward pass: get both the language model logits and policy values
            logits, policy_values = model(input_ids, dropout_rate=dropout_rate)
            logits = logits.view(-1, vocab_size)
            
            # Compute Cross Entropy Loss (CE) for language modeling
            criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
            ce_loss = criterion(logits, labels.view(-1))
    
            # Compute policy loss (using Gumbel-Softmax and FK reward advantage)
            probs = F.softmax(logits, dim=-1)
            sampled_tokens = torch.argmax(probs, dim=-1)  # Convert to discrete tokens
            
            # Decode the sampled tokens to text
            sampled_tokens = sampled_tokens.view(batch_size, seq_length)
            decoded_texts = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in sampled_tokens]
    
            # Compute FK reward based on the generated text
            fk_rewards = compute_fk_reward(decoded_texts)
            fk_ease = fk_rewards.mean().item()
    
            # Update EMA of rewards
            mean_rewards = fk_rewards.mean()
            ema = beta * ema + (1 - beta) * mean_rewards  # Update the EMA of rewards
    
            # Compute the baseline using the updated EMA
            baseline = ema
    
            # Compute the advantage by subtracting the baseline (EMA) from the FK rewards
            advantage = fk_rewards - baseline
    
            # Compute policy gradient loss using policy values
            policy_log_probs = torch.log_softmax(policy_values, dim=-1)
            policy_loss = -(policy_log_probs * advantage).mean()  # Negative log-likelihood weighted by advantage
    
            # Total loss: weighted sum of CE loss and policy loss
            alpha = 0.1  # Weighting factor between CE and RL losses
            total_loss_batch = ce_loss + alpha * policy_loss
    
            # Normalize loss for gradient accumulation
            total_loss_batch = total_loss_batch / accumulation_steps
            total_loss_batch.backward()
    
            accumulated_loss += total_loss_batch.item() * accumulation_steps
            total_fk_ease += fk_ease
            num_batches += 1
    
            accumulation_counter += 1
    
            if accumulation_counter % accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                accumulation_counter = 0
    
            loss_history.append(total_loss_batch.item() * accumulation_steps)
            avg_loss = accumulated_loss / num_batches
            avg_fk_ease = total_fk_ease / num_batches
            progress_bar.set_postfix(
                loss=total_loss_batch.item() * accumulation_steps, 
                avg_loss=avg_loss,
                fk_ease=fk_ease,
                avg_fk_ease=avg_fk_ease
            )
            total_batches += 1
    
            if total_batches % rotation_interval == 0:
                freeze_all_blocks(model)
                block_pair_manager.rotate_block_pair()
                unfreeze_block_pair(model, block_pair_manager.get_current_pair())
    
            if (batch_idx + 1) % checkpoint_interval == 0:
                # Save checkpoints
                save_model_checkpoint(model, epoch, batch_idx)
                save_optimizer_checkpoint(optimizer, epoch, batch_idx)
                save_scheduler_checkpoint(scheduler, epoch, batch_idx)
                save_training_state(epoch, batch_idx, loss_history)
    
                # Generate and print a sample output
                model.eval()
                with torch.no_grad():
                    input_ids = input_ids[:, :50]
                    sample_outputs, _ = model(input_ids, dropout_rate=dropout_rate)
                    sample_outputs = torch.argmax(sample_outputs, dim=-1)
                    decoded_input = tokenizer.decode(input_ids[0], skip_special_tokens=True)
                    decoded_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
                    print(f"Batch {batch_idx + 1}: Input Text: {decoded_input} Decoded Text: {decoded_text}")
                model.train()
    
        if accumulation_counter > 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
    
        save_model_checkpoint(model, epoch, batch_idx)
        save_optimizer_checkpoint(optimizer, epoch, batch_idx)
        save_scheduler_checkpoint(scheduler, epoch, batch_idx)
        save_training_state(epoch, batch_idx, loss_history)
        print(f"Epoch {epoch + 1} completed. Avg Loss: {avg_loss:.4f}, EMA Ease: {ema:.4f}")
    

# Run the Modal function
if __name__ == "__main__":
    with modal.enable_output():
        with app.run():
            run_training()

'''

# Specify the file name
file_name = "getting_started.py"

# Write the content to the file
with open(file_name, "w") as file:
    file.write(content)

print(f"Python file '{file_name}' has been created successfully.")


Python file 'getting_started.py' has been created successfully.


In [1]:
!pip install einops textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.9 MB/s eta 0:00:00a 0:00:01


In [ ]:
    import torch
    from numba import cuda, float64
    import math
    import numpy as np
    import textstat
    import torch
    import torch.nn.functional as F
    from torch.distributions import Categorical
    # Ensure that Numba uses the same device as PyTorch
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # -------------------------------
    # CUDA Kernels
    # -------------------------------
    @cuda.jit
    def fused_scan_fwd_kernel(u, delta, A, B, C, D, output, b, l, d_in, n):
        """
        Forward pass kernel implementing Blelloch's inclusive scan.
        """
        shared_mem = cuda.shared.array(shape=512, dtype=float64)
        input_x = shared_mem[:256]
        scan_x = shared_mem[256:]
        idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        total = b * l * d_in
        if idx >= total:
            return
        dim = idx % d_in
        seq = (idx // d_in) % l
        batch = idx // (l * d_in)
        u_val = u[batch, seq, dim]
        delta_val = delta[batch, seq, dim]
        # Initialize scan_x to zero
        for j in range(n):
            if j < n:
                scan_x[j] = 0.0
        cuda.syncthreads()
        # Compute scan_x[j] = exp(delta * A[j]) * scan_x[j] + delta * B[j] * u
        for j in range(n):
            if j < n:
                scan_x[j] = math.exp(delta_val * A[dim, j]) * scan_x[j] + delta_val * B[batch, seq, j] * u_val
        cuda.syncthreads()
        # Store original scan_x before scan for inclusive scan
        for j in range(n):
            if j < n:
                input_x[j] = scan_x[j]
        cuda.syncthreads()
        # Perform Blelloch's exclusive scan on scan_x
        step = 1
        while step < n:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                scan_x[index] += scan_x[index - step]
            cuda.syncthreads()
            step *= 2
        if cuda.threadIdx.x == 0 and n > 0:
            scan_x[n - 1] = 0.0
        cuda.syncthreads()
        step = n // 2
        while step >= 1:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                t = scan_x[index - step]
                scan_x[index - step] = scan_x[index]
                scan_x[index] += t
            cuda.syncthreads()
            step //= 2
        # Inclusive scan: add input_x[j] to scan_x[j]
        for j in range(n):
            if j < n:
                scan_x[j] += input_x[j]
        cuda.syncthreads()
        # Now scan_x contains the inclusive scan result
        # Compute y = sum(scan_x[j] * C[j]) + u * D[dim]
        y = 0.0
        for j in range(n):
            if j < n:
                y += scan_x[j] * C[batch, seq, j]
        y += u_val * D[dim]
        output[batch, seq, dim] = y
    @cuda.jit
    def fused_scan_bwd_kernel(u, delta, A, B, C, D, grad_output, grad_u, grad_delta,
                              grad_A, grad_B, grad_C, grad_D, b, l, d_in, n):
        """
        Backward pass kernel implementing Blelloch's inclusive scan and reverse scan for gradient computations.
        """
        # Allocate shared memory: double the size for input and scan
        shared_mem = cuda.shared.array(shape=1024, dtype=float64)  # Adjust size if needed
        input_x = shared_mem[:256]
        scan_x = shared_mem[256:512]
        grad_shared_A = shared_mem[512:768]
        grad_shared_B = shared_mem[768:1024]
        # Initialize shared gradients to zero
        for j in range(n):
            if j < n:
                grad_shared_A[j] = 0.0
                grad_shared_B[j] = 0.0
        cuda.syncthreads()
        idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
        total = b * l * d_in
        if idx >= total:
            return
        dim = idx % d_in
        seq = (idx // d_in) % l
        batch = idx // (l * d_in)
        u_val = u[batch, seq, dim]
        delta_val = delta[batch, seq, dim]
        y_grad = grad_output[batch, seq, dim]
        # Initialize scan_x to zero
        for j in range(n):
            if j < n:
                scan_x[j] = 0.0
        cuda.syncthreads()
        # Recompute scan_x as in forward pass
        for j in range(n):
            if j < n:
                scan_x[j] = math.exp(delta_val * A[dim, j]) * scan_x[j] + delta_val * B[batch, seq, j] * u_val
        cuda.syncthreads()
        # Store original scan_x before scan for inclusive scan
        for j in range(n):
            if j < n:
                input_x[j] = scan_x[j]
        cuda.syncthreads()
        # Perform Blelloch's exclusive scan on scan_x
        step = 1
        while step < n:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                scan_x[index] += scan_x[index - step]
            cuda.syncthreads()
            step *= 2
        if cuda.threadIdx.x == 0 and n > 0:
            scan_x[n - 1] = 0.0
        cuda.syncthreads()
        step = n // 2
        while step >= 1:
            index = 2 * step * cuda.threadIdx.x + (2 * step - 1)
            if index < n:
                t = scan_x[index - step]
                scan_x[index - step] = scan_x[index]
                scan_x[index] += t
            cuda.syncthreads()
            step //= 2
        # Inclusive scan: add input_x[j] to scan_x[j]
        for j in range(n):
            if j < n:
                scan_x[j] += input_x[j]
        cuda.syncthreads()
        # Reverse scan to propagate gradients
        # Initialize gradient accumulation for A and B
        for j in range(n):
            if j < n:
                grad_shared_A[j] = 0.0
                grad_shared_B[j] = 0.0
        cuda.syncthreads()
        # Compute gradients
        for j in range(n):
            if j < n:
                # Gradient w.r.t C[j]
                grad_C_val = scan_x[j] * y_grad
                grad_C[batch, seq, j] += grad_C_val
                # Gradient w.r.t A[j]
                grad_A_val = delta_val * math.exp(delta_val * A[dim, j]) * scan_x[j] * y_grad
                grad_shared_A[j] += grad_A_val
                # Gradient w.r.t B[j]
                grad_B_val = delta_val * u_val * y_grad
                grad_shared_B[j] += grad_B_val
                # Gradient w.r.t D[dim]
                grad_D[dim] += u_val * y_grad
        # Now, accumulate shared gradients for A and B
        for j in range(n):
            if j < n:
                A_grad = grad_shared_A[j]
                B_grad = grad_shared_B[j]
                grad_A[dim, j] += A_grad
                grad_B[batch, seq, j] += B_grad
        # Gradient w.r.t delta
        for j in range(n):
            if j < n:
                grad_delta_val = y_grad * (A[dim, j] * math.exp(delta_val * A[dim, j]) * scan_x[j] +
                                            B[batch, seq, j] * u_val)
                grad_delta[batch, seq, dim] += grad_delta_val
        # Gradient w.r.t u
        grad_u_val = 0.0
        for j in range(n):
            if j < n:
                grad_u_val += delta_val * B[batch, seq, j] * C[batch, seq, j] * y_grad
        grad_u[batch, seq, dim] += grad_u_val
    # -------------------------------
    # PyTorch Autograd Function
    # -------------------------------
    class FusedScanFunction(torch.autograd.Function):
        @staticmethod
        def forward(ctx, u, delta, A, B, C, D):
            """
            Forward pass using the custom CUDA kernel.
            """
            # Detach inputs to prevent PyTorch from tracking gradients through them
            u_detached = u.detach().contiguous().double().to(device)
            delta_detached = delta.detach().contiguous().double().to(device)
            A_detached = A.detach().contiguous().double().to(device)
            B_detached = B.detach().contiguous().double().to(device)
            C_detached = C.detach().contiguous().double().to(device)
            D_detached = D.detach().contiguous().double().to(device)
            b, l, d_in = u_detached.shape
            n = A_detached.shape[1]
            # Allocate output tensor
            output = torch.zeros_like(u_detached, device=device, dtype=torch.float64)
            # Define grid and block dimensions
            threads_per_block = 256
            blocks_per_grid = (b * l * d_in + (threads_per_block - 1)) // threads_per_block
            # Launch the forward kernel
            fused_scan_fwd_kernel[blocks_per_grid, threads_per_block](
                cuda.as_cuda_array(u_detached),
                cuda.as_cuda_array(delta_detached),
                cuda.as_cuda_array(A_detached),
                cuda.as_cuda_array(B_detached),
                cuda.as_cuda_array(C_detached),
                cuda.as_cuda_array(D_detached),
                cuda.as_cuda_array(output),
                b, l, d_in, n
            )
            # Save context for backward
            ctx.save_for_backward(u_detached, delta_detached, A_detached, B_detached, C_detached, D_detached)
            ctx.n = n
            ctx.b = b
            ctx.l = l
            ctx.d_in = d_in
            return output
        @staticmethod
        def backward(ctx, grad_output):
            u_detached, delta_detached, A_detached, B_detached, C_detached, D_detached = ctx.saved_tensors
            n = ctx.n
            b = ctx.b
            l = ctx.l
            d_in = ctx.d_in
            grad_output_detached = grad_output.contiguous().double().to(device)
            # Allocate gradient tensors
            grad_u = torch.zeros_like(u_detached, device=device, dtype=torch.float64)
            grad_delta = torch.zeros_like(delta_detached, device=device, dtype=torch.float64)
            grad_A = torch.zeros_like(A_detached, device=device, dtype=torch.float64)
            grad_B = torch.zeros_like(B_detached, device=device, dtype=torch.float64)
            grad_C = torch.zeros_like(C_detached, device=device, dtype=torch.float64)
            grad_D = torch.zeros_like(D_detached, device=device, dtype=torch.float64)
            # Define grid and block dimensions
            threads_per_block = 256
            blocks_per_grid = (b * l * d_in + (threads_per_block - 1)) // threads_per_block
            # Launch the backward kernel
            fused_scan_bwd_kernel[blocks_per_grid, threads_per_block](
                cuda.as_cuda_array(u_detached),
                cuda.as_cuda_array(delta_detached),
                cuda.as_cuda_array(A_detached),
                cuda.as_cuda_array(B_detached),
                cuda.as_cuda_array(C_detached),
                cuda.as_cuda_array(D_detached),
                cuda.as_cuda_array(grad_output_detached),
                cuda.as_cuda_array(grad_u),
                cuda.as_cuda_array(grad_delta),
                cuda.as_cuda_array(grad_A),
                cuda.as_cuda_array(grad_B),
                cuda.as_cuda_array(grad_C),
                cuda.as_cuda_array(grad_D),
                b, l, d_in, n
            )
            return grad_u, grad_delta, grad_A, grad_B, grad_C, grad_D
    # -------------------------------
    # Custom PyTorch Module
    # -------------------------------
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F
    from torch.autograd import Function
    from torch.utils.checkpoint import checkpoint
    import numpy as np
    from numba import cuda
    import math
    from einops import repeat
    import sys
    from torch.optim.lr_scheduler import CosineAnnealingLR
    import os
    import math
    import random
    import shutil
    import re
    from collections import Counter
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.cpp_extension import load
    from torch.utils.checkpoint import checkpoint
    from safetensors.torch import save_model, load_model
    from torch.nn.utils.rnn import pad_sequence
    from torch.optim import Optimizer
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from torch.autograd import Function
    from einops import repeat
    from transformers import GPT2Tokenizer
    from datasets import load_dataset
    from tqdm import tqdm
    # ============================
    # CUDA Kernels
    # ============================
    # Wrapper function to launch the forward kernel
    def fused_scan_fwd(u, delta, A, B, C, D):
        b, l, d_in = u.shape
        n = A.shape[1]
        output = cuda.device_array_like(u)
        threads_per_block = 256
        blocks_per_grid = (b * l * d_in + threads_per_block - 1) // threads_per_block
        blocks_per_grid = max(blocks_per_grid, 2)  # You can adjust this based on performance
        # shared_mem_size is not needed since we're using fixed shared memory
        fused_scan_fwd_kernel[blocks_per_grid, threads_per_block](
            u, delta, A, B, C, D, output, b, l, d_in, n
        )
        return output.copy_to_host()
    # Wrapper function to launch the backward kernel
    def fused_scan_bwd(u, delta, A, B, C, D, grad_output):
        b, l, d_in = u.shape
        n = A.shape[1]
        grad_u = cuda.device_array_like(u)
        grad_delta = cuda.device_array_like(delta)
        grad_A = cuda.device_array_like(A)
        grad_B = cuda.device_array_like(B)
        grad_C = cuda.device_array_like(C)
        grad_D = cuda.device_array_like(D)
        threads_per_block = 256
        blocks_per_grid = (b * l * d_in + threads_per_block - 1) // threads_per_block
        blocks_per_grid = max(blocks_per_grid, 2)  # You can adjust this based on performance
        # shared_mem_size is not needed since we're using fixed shared memory
        fused_scan_bwd_kernel[blocks_per_grid, threads_per_block](
            u, delta, A, B, C, D, grad_output, grad_u, grad_delta, grad_A, grad_B, grad_C, grad_D,
            b, l, d_in, n
        )
        return grad_u.copy_to_host(), grad_delta.copy_to_host(), grad_A.copy_to_host(), grad_B.copy_to_host(), grad_C.copy_to_host(), grad_D.copy_to_host()
    # ============================
    # Custom Layers and Functions
    # ============================
    def compute_dropout_rate(batch_idx, total_batches, max_dropout=0.4):
        """Compute dropout rate linearly increasing from 0 to max_dropout."""
        return min(max_dropout * (batch_idx / total_batches), max_dropout)
    class RMSNorm(nn.Module):
        def __init__(self, d_model: int, eps: float = 1e-5, use_mup: bool = False):
            super().__init__()
            self.use_mup = use_mup
            self.eps = eps
            # RMSNorm gains prevent muTransfer (section 4.2.3)
            if not use_mup:
                self.weight = nn.Parameter(torch.ones(d_model))
        def forward(self, x):
            output = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
            if not self.use_mup:
                return output * self.weight
            else:
                return output
    class SelectiveScanFunction(Function):
        @staticmethod
        def forward(ctx, u, delta, A_log, B, C, D):
            A = -torch.exp(A_log.float())
            # Ensure all tensors are contiguous and on the correct device
            u_contig = u.contiguous()
            delta_contig = delta.contiguous()
            A_contig = A.contiguous()
            B_contig = B.contiguous()
            C_contig = C.contiguous()
            D_contig = D.contiguous()
            # Move to Numba-compatible device arrays
            u_device = cuda.to_device(u_contig.cpu().numpy())
            delta_device = cuda.to_device(delta_contig.cpu().numpy())
            A_device = cuda.to_device(A_contig.cpu().numpy())
            B_device = cuda.to_device(B_contig.cpu().numpy())
            C_device = cuda.to_device(C_contig.cpu().numpy())
            D_device = cuda.to_device(D_contig.cpu().numpy())
            # Launch the forward kernel
            output_host = fused_scan_fwd(u_device, delta_device, A_device, B_device, C_device, D_device)
            # Convert output back to torch tensor
            output = torch.from_numpy(output_host).to(u.device)
            ctx.save_for_backward(u, delta, A_log, B, C, D)
            return output
        @staticmethod
        def backward(ctx, grad_output):
            u, delta, A_log, B, C, D = ctx.saved_tensors
            A = -torch.exp(A_log)
            # Move tensors to CPU and convert to NumPy
            u_cpu = u.contiguous().cpu().numpy()
            delta_cpu = delta.contiguous().cpu().numpy()
            A_cpu = A.contiguous().cpu().numpy()
            B_cpu = B.contiguous().cpu().numpy()
            C_cpu = C.contiguous().cpu().numpy()
            D_cpu = D.contiguous().cpu().numpy()
            grad_output_cpu = grad_output.contiguous().cpu().numpy()
            # Move to Numba-compatible device arrays
            u_device = cuda.to_device(u_cpu)
            delta_device = cuda.to_device(delta_cpu)
            A_device = cuda.to_device(A_cpu)
            B_device = cuda.to_device(B_cpu)
            C_device = cuda.to_device(C_cpu)
            D_device = cuda.to_device(D_cpu)
            grad_output_device = cuda.to_device(grad_output_cpu)
            # Launch the backward kernel
            grad_u_host, grad_delta_host, grad_A_host, grad_B_host, grad_C_host, grad_D_host = fused_scan_bwd(
                u_device, delta_device, A_device, B_device, C_device, D_device, grad_output_device
            )
            # Convert gradients back to torch tensors
            grad_u = torch.from_numpy(grad_u_host).to(u.device)
            grad_delta = torch.from_numpy(grad_delta_host).to(delta.device)
            grad_A = torch.from_numpy(grad_A_host).to(A.device)
            grad_B = torch.from_numpy(grad_B_host).to(B.device)
            grad_C = torch.from_numpy(grad_C_host).to(C.device)
            grad_D = torch.from_numpy(grad_D_host).to(D.device)
            # Compute grad_A_log
            grad_A_log = grad_A * (-torch.exp(A_log))
            return grad_u, grad_delta, grad_A_log, grad_B, grad_C, grad_D
    class SelectiveScanModel(nn.Module):
        def __init__(self, d_in, d_state, dt_rank):
            super().__init__()
            self.norm = RMSNorm(d_in)
            self.d_state = d_state
            self.dt_rank = dt_rank
            A = repeat(
                torch.arange(1, self.d_state + 1, dtype=torch.float32),
                "n -> d n",
                d=d_in,
            ).contiguous()
            A_log = torch.log(A)
            self.A_log = nn.Parameter(A_log)
            self.D = nn.Parameter(torch.ones(d_in).cuda())
            self.x_proj = nn.Linear(d_in, dt_rank + d_state * 2, bias=True).cuda()
            self.dt_proj = nn.Linear(dt_rank, d_in, bias=True).cuda()
            self.output_layer = nn.Linear(d_in, d_in).cuda()
        def forward(self, x):
            b, l, d_in = x.shape
            D = self.D
            x_dbl = self.x_proj(x)
            delta, B, C = x_dbl.split([self.dt_rank, self.d_state, self.d_state], dim=-1)
            delta = F.softplus(self.dt_proj(delta))
            y = self.norm(SelectiveScanFunction.apply(x, delta, self.A_log, B, C, D))
            return self.output_layer(y)
    class MambaBlock(nn.Module):
        def __init__(self, d_model, d_state=16, dt_rank=16, d_conv=4, expand=2, conv_bias=True, bias=False):
            super().__init__()
            self.d_model = d_model
            self.d_state = d_state
            self.d_conv = d_conv
            self.expand = expand
            self.d_inner = int(self.expand * self.d_model)
            self.dt_rank = dt_rank
            self.in_proj = nn.Linear(self.d_model, self.d_inner * 2, bias=bias)
            self.conv1d = nn.Sequential(
                nn.Conv1d(in_channels=self.d_inner, out_channels=self.d_inner, kernel_size=d_conv, groups=self.d_inner, bias=conv_bias),
                nn.Conv1d(in_channels=self.d_inner, out_channels=self.d_inner, kernel_size=1, bias=conv_bias)
            )
            self.activation = nn.SiLU()
            self.ssm_model = SelectiveScanModel(self.d_inner, self.d_state, self.dt_rank)
            self.out_proj = nn.Linear(self.d_inner, self.d_model, bias=bias)
            self.norm = RMSNorm(self.d_model)
        def forward(self, x):
            b, l, d = x.shape
            x_proj = self.in_proj(x)
            x, res = x_proj.chunk(2, dim=-1)
            x = x.transpose(1, 2)
            x = F.pad(x, (self.d_conv - 1, 0))
            x = self.conv1d(x)[:, :, :l]
            x = x.transpose(1, 2)
            x = self.activation(x)
            y = self.ssm_model(x)
            y = y * self.activation(res)
            y = self.norm(self.out_proj(y))
            return y
    class BlockStack(nn.Module):
        def __init__(self, num_blocks, d_model, d_state, dt_rank):
            super().__init__()
            self.blocks = nn.ModuleList([MambaBlock(d_model, d_state, dt_rank) for _ in range(num_blocks)])
        def forward(self, x, dropout_rate=None):
            x_res = x
            for i in range(len(self.blocks)):
                x = self.blocks[i](x)
            return x * x_res
    class FeedForwardLayer(nn.Module):
        def __init__(self, embed_dim, d_model, hidden_dim, window_size=4):
            """
            Initializes the FeedForwardLayer with windowed masking.
            Args:
                d_model (int): Dimension of the input and output features.
                hidden_dim (int): Dimension of the hidden layer.
                window_size (int): Size of the window around each position.
            """
            super().__init__()
            self.fc1 = nn.Linear(embed_dim, hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, d_model)
            self.activation = nn.SiLU()
            self.norm = RMSNorm(d_model)
        def forward(self, x):
            """
            Forward pass of the FeedForwardLayer with windowed masking.
            Args:
                x (torch.Tensor): Input tensor of shape (batch_size, seq_length, d_model).
            Returns:
                torch.Tensor: Output tensor of shape (batch_size, seq_length, d_model).
            """
            # Manually apply the windowed mask to the weights of the first linear layer
            masked_weight = self.fc1.weight  # Element-wise multiplication
            masked_bias = self.fc1.bias
            # Perform the masked linear transformation
            x = torch.matmul(x, masked_weight.t()) + masked_bias
            x = self.activation(x)
            x = self.fc2(x)
            x = self.norm(x)
            return x
    class HeadModel(nn.Module):
        def __init__(self, vocab_size, embed_dim, d_state, dt_rank, d_model, num_blocks=1, num_matrices=2):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embed_dim)
            self.block_stacks = nn.ModuleList([
                BlockStack(num_blocks, embed_dim // num_matrices, d_state, dt_rank)
                for _ in range(num_matrices)
            ])
            self.activation = nn.SiLU()
            self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
            self.policy_head = nn.Linear(d_model, 1, bias=False)  # New head for policy output (scalar for each token)
            self.norm = nn.LayerNorm(d_model)
            hidden_dim = d_model * 2
            self.feed_forward = FeedForwardLayer(embed_dim, d_model, hidden_dim)
            self.num_matrices = num_matrices
    
        def forward(self, input_ids, dropout_rate=None):
            x = self.embedding(input_ids)
            e = x
            split_dim = x.shape[2] // self.num_matrices
            x_splits = torch.split(x, split_dim, dim=2)
            processed_splits = []
            for i, feature_matrix in enumerate(x_splits):
                block_stack = self.block_stacks[i]
                processed_matrix = checkpoint(block_stack, feature_matrix, use_reentrant=False)
                processed_splits.append(processed_matrix)
            x = torch.cat(processed_splits, dim=2)
            x = self.feed_forward(x + e)
            x = self.norm(x)
            
            logits = self.lm_head(x)  # Output for language modeling
            policy_values = self.policy_head(x).squeeze(-1)  # Output policy values (scalar for each token)
            
            return logits, policy_values
    # ============================
    # 3. Data Preparation
    # ============================
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})
    dataset = load_dataset("LLM360/TxT360", split="train", streaming=True, trust_remote_code=True)
    import torch
    import textstat
    
    def compute_fk_reward(decoded_texts):
        """
        Compute Flesch-Kincaid readability scores as rewards for a batch of decoded texts.
    
        Args:
            decoded_texts (list of str): List of generated texts.
    
        Returns:
            torch.Tensor: FK readability scores normalized to be positive rewards using soft clamping.
        """
        rewards = []
        for text in decoded_texts:
            fk_score = textstat.flesch_kincaid_grade(text)
    
            # Convert fk_score and raw_reward to a tensor
            raw_reward = torch.tensor(fk_score/100, dtype=torch.float32).cuda()
    
            # Apply a sigmoid-based function to softly clamp between 0 and 10
            reward = 10 / (1 + torch.exp(-raw_reward / 2))  # Scaling to control the smoothness
    
            rewards.append(raw_reward)
    
        # Convert the list of rewards to a torch tensor for batch processing
        rewards = torch.stack(rewards)  # Stack the list into a single tensor
        return rewards
    def collate_fn(batch, tokenizer):
        tokenized_inputs = [item['tokenized_input'] for item in batch]
        tokenized_targets = [item['tokenized_target'] for item in batch]
        bos_token_id = tokenizer.bos_token_id
        eos_token_id = tokenizer.eos_token_id
        tokenized_inputs = [
            torch.cat([torch.tensor([bos_token_id]), tokenized_input, torch.tensor([eos_token_id])])
            for tokenized_input in tokenized_inputs
        ]
        tokenized_targets = [
            torch.cat([torch.tensor([bos_token_id]), tokenized_target, torch.tensor([eos_token_id])])
            for tokenized_target in tokenized_targets
        ]
        padded_inputs = pad_sequence(tokenized_inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
        padded_targets = pad_sequence(tokenized_targets, batch_first=True, padding_value=tokenizer.pad_token_id)
        return padded_inputs, padded_targets
    class BufferedStreamingDataLoader:
        def __init__(self, dataset, batch_size, buffer_size, collate_fn, tokenizer, shuffle_prob=0.3, mask_prob=0.5):
            self.dataset = dataset
            self.batch_size = batch_size
            self.buffer_size = buffer_size
            self.collate_fn = collate_fn
            self.tokenizer = tokenizer
            self.shuffle_prob = shuffle_prob
            self.mask_prob = mask_prob
        def _split_into_sentences(self, tokenized_input):
            sentence_end_tokens = {
                self.tokenizer.convert_tokens_to_ids('.'),
                self.tokenizer.convert_tokens_to_ids('!'),
                self.tokenizer.convert_tokens_to_ids('?')
            }
            sentences = []
            current_sentence = []
            for token in tokenized_input:
                current_sentence.append(token)
                if token in sentence_end_tokens:
                    sentences.append(current_sentence)
                    current_sentence = []
            if current_sentence:
                sentences.append(current_sentence)
            return sentences
        def _shuffle_sentences(self, tokenized_input):
            sentences = self._split_into_sentences(tokenized_input)
            random.shuffle(sentences)
            shuffled_input = [token for sentence in sentences for token in sentence]
            return shuffled_input
        def _apply_random_mask(self, tokenized_input):
            length = len(tokenized_input)
            if length > 1:
                mask_start = int(length * 0.33)
                mask_end = int(length * 0.66)
                random_mask_start = random.randint(mask_start, mask_end)
                mask_length = random.randint(int(0.33 * length), int(0.66 * length))
                mask_end_idx = length
                masked_input = tokenized_input.clone()
                mask_token_id = self.tokenizer.pad_token_id
                masked_input[random_mask_start:mask_end_idx] = mask_token_id
                return masked_input
            return tokenized_input
        def __iter__(self):
            buffer = []
            token_counter = Counter()
            for item in self.dataset:
                tokenized_input = self.tokenizer(
                    item['text'], return_tensors='pt', max_length=4096, padding=False, truncation=True
                )['input_ids'][0]
                tokenized_target = self.tokenizer(
                    item['text'], return_tensors='pt', max_length=4096, padding=False, truncation=True
                )['input_ids'][0]
                if random.random() < self.shuffle_prob:
                    shuffled_input = self._shuffle_sentences(tokenized_input.tolist())
                    tokenized_input = torch.tensor(shuffled_input)
                else:
                    tokenized_input = tokenized_input
                    if random.random() < self.mask_prob:
                        tokenized_input = self._apply_random_mask(tokenized_input)
                buffer.append({
                    'text':  item['text'],
                    'tokenized_input': tokenized_input,
                    'tokenized_target': tokenized_target
                })
                token_counter.update(tokenized_input.tolist())
                if len(buffer) == self.buffer_size:
                    buffer = sorted(buffer, key=lambda x: len(x['tokenized_input']), reverse=False)
                    for i in range(0, len(buffer), self.batch_size):
                        batch = buffer[i:i + self.batch_size]
                        yield self.collate_fn(batch, self.tokenizer), token_counter
                    buffer = []
                    token_counter = Counter()
            if buffer:
                buffer = sorted(buffer, key=lambda x: len(x['tokenized_input']), reverse=True)
                for i in range(0, len(buffer), self.batch_size):
                    batch = buffer[i:i + self.batch_size]
                    yield self.collate_fn(batch, self.tokenizer), token_counter
    # ============================
    # 4. Model Initialization
    # ============================
    def initialize_model_parameters(model):
        """ Initialize all trainable parameters in the model. """
        for name, param in model.named_parameters():
            if 'weight' in name:
                if len(param.shape) > 1:
                    nn.init.xavier_uniform_(param)
                else:
                    nn.init.normal_(param, 0.0, 0.02)
            elif 'bias' in name:
                nn.init.zeros_(param)
    batch_size = 1
    buffer_size = 100 * batch_size
    dataloader = BufferedStreamingDataLoader(
        dataset, batch_size=batch_size, buffer_size=buffer_size, collate_fn=collate_fn, tokenizer=tokenizer
    )
    vocab_size = tokenizer.vocab_size + 3
    embed_dim = 512
    d_state = 16
    d_model = 1024
    dt_rank = math.ceil(d_model / 16)
    model = HeadModel(vocab_size, embed_dim, d_state, dt_rank, d_model).cuda()
    initialize_model_parameters(model)
    model = nn.DataParallel(model)
    # ============================
    # 5. Optimizer and Scheduler
    # ============================
    import math
    import torch
    from torch.optim import Optimizer
    import math
    import torch
    from torch.optim import Optimizer
    from torch.autograd import Variable
    class AdEMAMix(Optimizer):
        def __init__(self, params, lr=1e-3, betas=(0.95, 0.979, 0.9995), eps=1e-7,
                      weight_decay=0.001, alpha=10.0, T_alpha_beta3=300000, max_steps=300000,
                      beta_opposite=0.99, nystrom_rank=200, nystrom_interval=50):
            if not 0.0 <= lr:
                raise ValueError(f"Invalid learning rate: {lr}")
            if not 0.0 <= eps:
                raise ValueError(f"Invalid epsilon value: {eps}")
            assert len(betas) == 3, f"Invalid beta parameters: {betas}, expected 3"
            assert all(0.0 <= beta < 1.0 for beta in betas), f"Invalid beta parameters: {betas}"
            if not 0.0 <= weight_decay:
                raise ValueError(f"Invalid weight_decay value: {weight_decay}")
            defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                            alpha=alpha, T_alpha_beta3=T_alpha_beta3)
            super(AdEMAMix, self).__init__(params, defaults)
            self.global_step = 0
            self.beta1, self.beta2, self.beta3 = betas
            self.alpha = alpha
            self.T_alpha_beta3 = T_alpha_beta3
            self.max_steps = max_steps
            self.beta_opposite = beta_opposite
            # Nyström parameters
            self.nystrom_rank = nystrom_rank
            self.nystrom_interval = nystrom_interval
            self.gradient_samples = []  # Buffer to store gradient samples
            self.nystrom_matrix = None  # Approximated covariance matrix
            # Precompute beta powers
            self.beta1_pows = [1.0]
            self.beta2_pows = [1.0]
            self.beta3_pows = [1.0]
            self.beta_opposite_pows = [1.0]
            for step in range(1, max_steps + 1):
                self.beta1_pows.append(self.beta1_pows[-1] * self.beta1)
                self.beta2_pows.append(self.beta2_pows[-1] * self.beta2)
                self.beta_opposite_pows.append(self.beta_opposite_pows[-1] * self.beta_opposite)
                if self.T_alpha_beta3 is not None:
                    alpha_t = min(step * self.alpha / self.T_alpha_beta3, self.alpha)
                    exponent = (math.log(self.beta1) * math.log(self.beta3)) / (
                        (1 - step / self.T_alpha_beta3) * math.log(self.beta3) +
                        (step / self.T_alpha_beta3) * math.log(self.beta1)
                    )
                    beta3_t = min(math.exp(exponent), self.beta3)
                else:
                    beta3_t = self.beta3
                self.beta3_pows.append(beta3_t)
        def __setstate__(self, state):
            super(AdEMAMix, self).__setstate__(state)
        @torch.no_grad()
        def step(self, closure=None):
            loss = None
            if closure is not None:
                with torch.enable_grad():
                    loss = closure()
            self.global_step += 1  # Increment global step once per step
            for group in self.param_groups:
                params_with_grad = []
                grads = []
                exp_avgs = []
                exp_avg_sqs = []
                exp_avg_slow = []
                exp_avg_opposite = []
                for p in group['params']:
                    if p.grad is not None:
                        if p.grad.is_sparse:
                            raise RuntimeError('AdEMAMixNyström does not support sparse gradients')
                        params_with_grad.append(p)
                        grads.append(p.grad)
                        state = self.state[p]
                        if len(state) == 0:
                            state['step'] = 0
                            state['exp_avg'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_sq'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_slow'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                            state['exp_avg_opposite'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                        exp_avgs.append(state['exp_avg'])
                        exp_avg_sqs.append(state['exp_avg_sq'])
                        exp_avg_slow.append(state['exp_avg_slow'])
                        exp_avg_opposite.append(state['exp_avg_opposite'])
                        state['step'] += 1
                beta1, beta2, beta3 = group['betas']
                alpha = group['alpha']
                T_alpha_beta3 = group['T_alpha_beta3']
                step = self.global_step
                if step <= self.max_steps:
                    bias_correction1 = 1 - self.beta1_pows[step]
                    bias_correction2 = 1 - self.beta2_pows[step]
                    bias_correction_opposite = 1 - self.beta_opposite_pows[step]
                    if T_alpha_beta3 is not None:
                        beta3_t = self.beta3_pows[step]
                    else:
                        beta3_t = beta3
                else:
                    bias_correction1 = 1 - self.beta1 ** step
                    bias_correction2 = 1 - self.beta2 ** step
                    bias_correction_opposite = 1 - self.beta_opposite ** step
                    if T_alpha_beta3 is not None:
                        alpha_t = min(step * alpha / T_alpha_beta3, alpha)
                        beta3_t = min(math.exp(math.log(beta1) * math.log(beta3) / 
                                      ((1 - step / T_alpha_beta3) * math.log(beta3) + 
                                        (step / T_alpha_beta3) * math.log(beta1))), self.beta3)
                    else:
                        beta3_t = beta3
                # Update Nyström samples
                if step % self.nystrom_interval == 0:
                    self._update_nystrom(params_with_grad, grads)
                # Compute preconditioning matrix using Nyström approximation
                if self.nystrom_matrix is not None:
                    precondition_matrix = self._compute_preconditioning()
                else:
                    precondition_matrix = None
                # Update parameters
                for i, param in enumerate(params_with_grad):
                    grad = grads[i]
                    exp_avg = exp_avgs[i]
                    exp_avg_sq = exp_avg_sqs[i]
                    exp_avg_slow_i = exp_avg_slow[i]
                    exp_avg_opposite_i = exp_avg_opposite[i]
                    # Update biased first moment estimate
                    exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)
                    # Update opposing EMA
                    exp_avg_opposite_i.mul_(self.beta_opposite).add_(grad.neg(), alpha=1 - self.beta_opposite)
                    # Update biased second moment estimate
                    exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)
                    # Update slow EMA
                    exp_avg_slow_i.mul_(beta3_t).add_(grad, alpha=1 - beta3_t)
                    # Combine primary and opposing EMAs
                    combined_momentum = exp_avg + alpha * exp_avg_slow_i + exp_avg_opposite_i
                    denom = (exp_avg_sq.sqrt() / math.sqrt(bias_correction2)).add_(group['eps'])
                    step_size = group['lr'] / bias_correction1
                    if group['weight_decay'] != 0:
                        param.add_(param, alpha=-group['weight_decay'] * group['lr'])
                    # Apply preconditioning if available
                    if precondition_matrix is not None:
                        combined_momentum = precondition_matrix.matmul(combined_momentum.view(-1, 1)).view_as(combined_momentum)
                    param.addcdiv_(combined_momentum, denom, value=-step_size)
            return loss
        def _update_nystrom(self, params, grads):
            """
            Update the gradient samples used for Nyström approximation.
            """
            # Flatten and concatenate all gradients into a single vector
            flat_grads = torch.cat([g.view(-1) for g in grads])
            flat_grads = flat_grads.cpu().detach()
            # Append to samples
            self.gradient_samples.append(flat_grads)
            # Maintain only the latest nystrom_rank samples
            if len(self.gradient_samples) > self.nystrom_rank:
                self.gradient_samples.pop(0)
            # Construct the Nyström matrix if enough samples are available
            if len(self.gradient_samples) == self.nystrom_rank:
                # Create a matrix where each column is a sampled gradient
                G = torch.stack(self.gradient_samples, dim=1)  # Shape: (num_params, nystrom_rank)
                # Compute the covariance matrix
                C = G @ G.t() / self.nystrom_rank  # Shape: (num_params, num_params)
                # Perform eigendecomposition
                eigenvalues, eigenvectors = torch.linalg.eigh(C)
                # Select the top k eigenvalues and eigenvectors
                idx = torch.argsort(eigenvalues, descending=True)[:self.nystrom_rank]
                self.nystrom_matrix = eigenvectors[:, idx] @ torch.diag(1.0 / torch.sqrt(eigenvalues[idx] + 1e-10)) @ eigenvectors[:, idx].t()
        def _compute_preconditioning(self):
            """
            Compute the preconditioning matrix using the Nyström approximation.
            """
            return self.nystrom_matrix.to(next(self.param_groups[0]['params']).device)
    optimizer = AdEMAMix(model.parameters(), lr=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=500000, eta_min=1e-6)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    # ============================
    # 6. Checkpointing Functions
    # ============================
    checkpoint_dir="/kaggle/working/"
    os.makedirs(checkpoint_dir, exist_ok=True)
    max_checkpoints = 5  # Maximum number of checkpoints to keep
    def save_model_checkpoint(model, epoch, batch_idx):
        model_dir = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}")
        os.makedirs(model_dir, exist_ok=True)
        # Save the entire model using safetensors to handle shared tensors
        model_to_save = model.module if isinstance(model, nn.DataParallel) else model
        save_model(model_to_save, os.path.join(model_dir, "model.safetensors"))
        cleanup_old_checkpoints()
    def save_optimizer_checkpoint(optimizer, epoch, batch_idx):
        optimizer_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "optimizer.pt")
        torch.save(optimizer.state_dict(), optimizer_state_file)
    def save_scheduler_checkpoint(scheduler, epoch, batch_idx):
        scheduler_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "scheduler.pt")
        torch.save(scheduler.state_dict(), scheduler_state_file)
    def save_training_state(epoch, batch_idx, loss_history):
        training_state_file = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}_batch_{batch_idx}", "training_state.pt")
        training_state = {
            'epoch': epoch,
            'batch_idx': batch_idx,
            'loss_history': loss_history
        }
        torch.save(training_state, training_state_file)
    def cleanup_old_checkpoints():
        checkpoints = sorted(
            [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))],
            key=lambda x: os.path.getctime(os.path.join(checkpoint_dir, x)),
            reverse=True
        )
        if len(checkpoints) > max_checkpoints:
            old_checkpoints = checkpoints[max_checkpoints:]
            for old_checkpoint in old_checkpoints:
                old_checkpoint_path = os.path.join(checkpoint_dir, old_checkpoint)
                shutil.rmtree(old_checkpoint_path)
    import os
    import re
    import torch
    from torch import nn
    from safetensors.torch import load_file  # Import the correct function to load state_dict
    from safetensors.torch import load_file  # Ensure this import is present
    def load_latest_checkpoint(model, optimizer, scheduler, checkpoint_dir="/kaggle/working/"):
        """
        Load the second from the last checkpoint from the specified checkpoint directory.
        Parameters:
        - model (torch.nn.Module): The model to load the state into.
        - optimizer (torch.optim.Optimizer): The optimizer to load the state into.
        - scheduler (torch.optim.lr_scheduler._LRScheduler): The scheduler to load the state into.
        - checkpoint_dir (str): Directory where checkpoints are stored.
        Returns:
        - epoch (int): The epoch number to resume from.
        - batch_idx (int): The batch index to resume from.
        - loss_history (list): History of loss values.
        - optimizer (torch.optim.Optimizer): Optimizer with loaded state.
        - scheduler (torch.optim.lr_scheduler._LRScheduler): Scheduler with loaded state.
        """
        # Ensure the checkpoint directory exists
        if not os.path.isdir(checkpoint_dir):
            print(f"Checkpoint directory '{checkpoint_dir}' does not exist. Starting training from scratch.")
            return 0, 0, [], optimizer, scheduler
        # List all checkpoint directories
        checkpoints = sorted(
            [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d))],
            key=lambda x: os.path.getctime(os.path.join(checkpoint_dir, x)),
            reverse=True
        )
        # Check if there are at least two checkpoints
        if len(checkpoints) < 2:
            if len(checkpoints) == 1:
                print("Only one checkpoint found. Loading the latest checkpoint.")
                latest_checkpoint = checkpoints[0]
            else:
                print("No checkpoints found. Starting training from scratch.")
                return 0, 0, [], optimizer, scheduler
        else:
            # Load the second from the last checkpoint
            latest_checkpoint = checkpoints[1]
            print(f"Resuming training from second latest checkpoint: {latest_checkpoint}")
        # Extract epoch and batch index from the checkpoint name
        match = re.match(r"checkpoint_epoch_(\d+)_batch_(\d+)", latest_checkpoint)
        if match:
            epoch = int(match.group(1))
            batch_idx = int(match.group(2))
        else:
            print("Checkpoint naming convention not recognized. Starting from scratch.")
            return 0, 0, [], optimizer, scheduler
        checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
        # Load the model state using safetensors
        model_to_load = model.module if isinstance(model, nn.DataParallel) else model
        model_state_path = os.path.join(checkpoint_path, "model.safetensors")
        if os.path.exists(model_state_path):
            load_model(model_to_load, model_state_path)
            print(f"Model state loaded from {model_state_path}.")
        else:
            print(f"Model state file '{model_state_path}' not found. Starting from scratch.")
            return 0, 0, [], optimizer, scheduler
        # Load optimizer state
        optimizer_state_file = os.path.join(checkpoint_path, "optimizer.pt")
        if os.path.exists(optimizer_state_file):
            optimizer.load_state_dict(torch.load(optimizer_state_file))
            print(f"Optimizer state loaded from {optimizer_state_file}.")
        else:
            print(f"Optimizer state file '{optimizer_state_file}' not found. Continuing without loading optimizer state.")
        # Load scheduler state
        scheduler_state_file = os.path.join(checkpoint_path, "scheduler.pt")
        if os.path.exists(scheduler_state_file):
            scheduler.load_state_dict(torch.load(scheduler_state_file))
            print(f"Scheduler state loaded from {scheduler_state_file}.")
        else:
            print(f"Scheduler state file '{scheduler_state_file}' not found. Continuing without loading scheduler state.")
        # Load training state (e.g., loss history)
        training_state_file = os.path.join(checkpoint_path, "training_state.pt")
        if os.path.exists(training_state_file):
            training_state = torch.load(training_state_file)
            loss_history = training_state.get('loss_history', [])
            print(f"Training state loaded from {training_state_file}.")
        else:
            loss_history = []
            print(f"Training state file '{training_state_file}' not found. Continuing without loading training state.")
        return 0, 0, loss_history, optimizer, scheduler
    # ============================
    # 7. Training Loop
    # ============================
    def freeze_all_blocks(model):
        if isinstance(model, torch.nn.DataParallel):
            block_stacks = model.module.block_stacks
        else:
            block_stacks = model.block_stacks
        # Freeze all blocks and corresponding mlps in each block stack
        for block_stack in block_stacks:
            for idx, block in enumerate(block_stack.blocks):
                # Freeze all parameters in the blocks
                for param in block.parameters():
                    param.requires_grad = False
    def unfreeze_block_pair(model, block_indices):
        if isinstance(model, torch.nn.DataParallel):
            block_stacks = model.module.block_stacks
        else:
            block_stacks = model.block_stacks
        # Unfreeze only the specified blocks and corresponding mlps in each block stack
        for block_stack in block_stacks:
            for idx in block_indices:
                # Unfreeze the block
                for param in block_stack.blocks[idx].parameters():
                    param.requires_grad = True
    class BlockPairManager:
        def __init__(self, num_blocks, pair_size=2):
            self.num_blocks = num_blocks
            self.pair_size = pair_size
            self.block_pairs = self.generate_block_pairs()
            self.current_pair_idx = 0
        def generate_block_pairs(self):
            block_pairs = []
            for i in range(0, self.num_blocks, self.pair_size):
                pair = tuple(range(i, min(i + self.pair_size, self.num_blocks)))
                block_pairs.append(pair)
            return block_pairs
        def get_current_pair(self):
            return self.block_pairs[self.current_pair_idx]
        def rotate_block_pair(self):
            self.current_pair_idx = (self.current_pair_idx + 1) % len(self.block_pairs)
        def reset(self):
            self.current_pair_idx = 0
    # Define accumulation steps
    accumulation_steps = 8  # Adjust based on your memory constraints
    # Initialize EMA of rewards (can be zero initially or set to the first batch's rewards)
    ema = 0.0  # Start at zero or initialize with the first reward batch
    beta = 0.99  # Smoothing factor for EMA
    num_epochs = 3
    checkpoint_interval = 200
    rotation_interval = 500
    total_batches = 0
    loss_history = []
    num_blocks = 1
    block_pair_manager = BlockPairManager(num_blocks=num_blocks, pair_size=1)
    start_epoch, start_batch_idx, loss_history, optimizer, scheduler = load_latest_checkpoint(model, optimizer, scheduler)
    max_sampling_p = 0.9
    # Gumbel-Softmax Sampling
    def sample_gumbel_softmax(logits, temperature=1.0, hard=False):
        """
        Performs Gumbel-Softmax sampling on the logits.
        
        Args:
            logits (torch.Tensor): Logits to sample from.
            temperature (float): Temperature parameter for Gumbel-Softmax.
            hard (bool): Whether to return hard (one-hot) samples or soft samples.
            
        Returns:
            torch.Tensor: Sampled tokens in the relaxed form, or one-hot encoded if hard=True.
        """
        # Sample from the Gumbel distribution
        gumbel_noise = -torch.log(-torch.log(torch.rand_like(logits) + 1e-10) + 1e-10)
        
        # Add Gumbel noise to the logits and apply softmax with temperature
        y_soft = F.softmax((logits + gumbel_noise) / temperature, dim=-1)
        
        if hard:
            # Straight-through trick to obtain discrete tokens
            y_hard = torch.argmax(y_soft, dim=-1)
            y_hard_one_hot = F.one_hot(y_hard, num_classes=logits.size(-1)).float()
            y = (y_hard_one_hot - y_soft).detach() + y_soft  # Differentiable discrete tokens
        else:
            y = y_soft
    
        return y
    
    # Inside your training loop
    # Inside your training loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_loss = 0
        total_fk_ease = 0
        num_batches = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}")
        freeze_all_blocks(model)
        unfreeze_block_pair(model, block_pair_manager.get_current_pair())
    
        accumulation_counter = 0
        accumulated_loss = 0.0
        optimizer.zero_grad()
    
        for batch_idx, ((input_ids, labels), freq) in enumerate(progress_bar):
            if epoch == start_epoch and batch_idx < start_batch_idx:
                continue
    
            input_ids = input_ids.cuda()
            input_ids = input_ids[:, :-1]
            labels = labels[:, 1:].contiguous().view(-1).cuda()
            batch_size, seq_length = input_ids.size()
            p = min(batch_idx / 18000.0 * max_sampling_p, max_sampling_p)
            dropout_rate = compute_dropout_rate(batch_idx + total_batches * epoch, 30000)
    
            # Forward pass: get both the language model logits and policy values
            logits, policy_values = model(input_ids, dropout_rate=dropout_rate)
            logits = logits.view(-1, vocab_size)
            
            # Compute Cross Entropy Loss (CE) for language modeling
            criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
            ce_loss = criterion(logits, labels.view(-1))
    
            # Compute policy loss (using Gumbel-Softmax and FK reward advantage)
            probs = F.softmax(logits, dim=-1)
            sampled_tokens = torch.argmax(probs, dim=-1)  # Convert to discrete tokens
            
            # Decode the sampled tokens to text
            sampled_tokens = sampled_tokens.view(batch_size, seq_length)
            decoded_texts = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in sampled_tokens]
    
            # Compute FK reward based on the generated text
            fk_rewards = compute_fk_reward(decoded_texts)
            fk_ease = fk_rewards.mean().item()
    
            # Update EMA of rewards
            mean_rewards = fk_rewards.mean()
            ema = beta * ema + (1 - beta) * mean_rewards  # Update the EMA of rewards
    
            # Compute the baseline using the updated EMA
            baseline = ema
    
            # Compute the advantage by subtracting the baseline (EMA) from the FK rewards
            advantage = fk_rewards - baseline
    
            # Compute policy gradient loss using policy values
            policy_log_probs = torch.log_softmax(policy_values, dim=-1)
            policy_loss = -(policy_log_probs * advantage).mean()  # Negative log-likelihood weighted by advantage
    
            # Total loss: weighted sum of CE loss and policy loss
            alpha = 0.01  # Weighting factor between CE and RL losses
            total_loss_batch = ce_loss + alpha * policy_loss
    
            # Normalize loss for gradient accumulation
            total_loss_batch = total_loss_batch / accumulation_steps
            total_loss_batch.backward()
    
            accumulated_loss += total_loss_batch.item() * accumulation_steps
            total_fk_ease += fk_ease
            num_batches += 1
    
            accumulation_counter += 1
    
            if accumulation_counter % accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                accumulation_counter = 0
    
            loss_history.append(total_loss_batch.item() * accumulation_steps)
            avg_loss = accumulated_loss / num_batches
            avg_fk_ease = total_fk_ease / num_batches
            progress_bar.set_postfix(
                loss=total_loss_batch.item() * accumulation_steps, 
                avg_loss=avg_loss,
                fk_ease=fk_ease,
                avg_fk_ease=avg_fk_ease
            )
            total_batches += 1
    
            if total_batches % rotation_interval == 0:
                freeze_all_blocks(model)
                block_pair_manager.rotate_block_pair()
                unfreeze_block_pair(model, block_pair_manager.get_current_pair())
    
            if (batch_idx + 1) % checkpoint_interval == 0:
                # Save checkpoints
                save_model_checkpoint(model, epoch, batch_idx)
                save_optimizer_checkpoint(optimizer, epoch, batch_idx)
                save_scheduler_checkpoint(scheduler, epoch, batch_idx)
                save_training_state(epoch, batch_idx, loss_history)
    
                # Generate and print a sample output
                model.eval()
                with torch.no_grad():
                    input_ids = input_ids[:, :50]
                    sample_outputs, _ = model(input_ids, dropout_rate=dropout_rate)
                    sample_outputs = torch.argmax(sample_outputs, dim=-1)
                    decoded_input = tokenizer.decode(input_ids[0], skip_special_tokens=True)
                    decoded_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
                    print(f"Batch {batch_idx + 1}: Input Text: {decoded_input} Decoded Text: {decoded_text}")
                model.train()
    
        if accumulation_counter > 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
    
        save_model_checkpoint(model, epoch, batch_idx)
        save_optimizer_checkpoint(optimizer, epoch, batch_idx)
        save_scheduler_checkpoint(scheduler, epoch, batch_idx)
        save_training_state(epoch, batch_idx, loss_history)
        print(f"Epoch {epoch + 1} completed. Avg Loss: {avg_loss:.4f}, EMA Ease: {ema:.4f}")


In [9]:
!pip install einops textstat 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.5 MB/s eta 0:00:0000:01


In [ ]:
!modal run --detach getting_started.py

✓ Initialized. View run at 
https://modal.com/apps/efloodproto/main/ap-dk1vDrYkTIO0ExbdT6iy4b
⠋ Initializing...
⠸ Creating objects...objects...
⠦ Creating objects...ggle/working/getting_started.py: Uploaded 0/1 files
⠏ Creating objects...ing/getting_started.py
├── 🔨 Created mount /kaggle/working/getting_started.py
⠏ Creating objects...ning...
├── 🔨 Created mount /kaggle/working/getting_started.py
└── 🔨 Created function run_training.
✓ Created objects.
├── 🔨 Created mount /kaggle/working/getting_started.py
└── 🔨 Created function run_training.
⠙ Worker assigned... View app at 
⠴ Worker assigned... View app at 
⠇ Running... View app at bdT6iy4b
⠙ Running... View app at bdT6iy4b
⠼ Running... View app at bdT6iy4b
⠧ Running... View app at bdT6iy4b
⠋ Running... View app at bdT6iy4b
⠸ Running... View app at bdT6iy4b
⠧ Running... View app at bdT6iy4b
⠋ Loading images (1 containers initializing)... View app at 
⠸ Loading images (1 containers initializing)... View app at 
⠦ Loading images (1 cont